This is a markdown cell. It describes the purpose of this notebook and its code.The other cells are coding cells. 

This notebook will extract the information from the scanned Manorial Records and populate excel files with them. 

In [2]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 20.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 19.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 10.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 10.7 MB/s eta 0:00:00


In [4]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00a 0:00:01


In [15]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 19.7 MB/s eta 0:00:0000:0100:01


In [35]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.2 MB/s eta 0:00:00:00:01


In [ ]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 4.4 MB/s eta 0:00:0000:01


In [1]:
import importlib
import subprocess
import sys

# List of required packages
required_packages = [
    "pandas",
    "PyPDF2",
    "pdfplumber",
    "openpyxl",
    "python-docx",  # install name different
]

# Install missing packages
for package in required_packages:
    try:
        importlib.import_module(package if package != "python-docx" else "docx")
    except ImportError:
        print(f"Installing missing package: {package}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("Packages installed")

Packages installed


In [2]:
# Import the pandas library with an alias 'pd' (this is a common convention)
import pandas as pd

# Import libraries for pdf reading
from PyPDF2 import PdfReader

# Better use this as it recognizes fonts: 
import pdfplumber

# Import openpyxl to use its functionality for working with Excel files
import openpyxl

# Extracting from word document 
import docx

#Import regular expressionsio
import re

In [8]:
####################### SCRIPT 1  ####################### 


# Function to check if a paragraph is a heading (bold text in this case)
def is_heading(paragraph):
    for run in paragraph.runs:
        if run.bold:
            return True
    return False

def split_into_sentences(paragraph):
   # Replace 's.' with 'shilling' when it follows a number
    paragraph = re.sub(r'(\d)s\.', r'\1 shilling', paragraph)
    # Replace standalone 's.' with 'shilling'
    paragraph = re.sub(r'\bs\.\b', 'shilling', paragraph)

    # Replace other abbreviations
    paragraph = paragraph.replace('d.', 'pence').replace('qrs', 'quarters').replace('bus.', 'bushels')

    # Define a regex pattern for sentence ending (. ; and In are recognized as sentence endings/beginnings)
    pattern = r'(?<!£)(?<!\s)(?<!\d\s)(?<!\d)[.](?!\d)(?!\s\d)(?!\s[a-z])(?= (?=[A-Z])|(?=$))|(?<=\s)In\s|;'

    sentences = re.split(pattern, paragraph)

    sentences = [sentence.strip() + (';' if sentence.endswith(';') else '.') for sentence in sentences[:-1]] + [sentences[-1].strip()]

    return sentences

#Extracting monetary amounts
# Updated regular expression pattern for extracting monetary amounts
# Adjusted to capture decimal values before "pence"
def extract_first_monetary_amount(sentence):
    # Initialize default monetary amounts
    monetary_amounts = {'Pounds': 0, 'Shillings': 0, 'Pence': 0}

    # Regex to capture the first coherent monetary group
    monetary_group_pattern = r'£(\d+(?:\.\d+)?)(?:\s(\d+(?:\.\d+)?)\s?shilling)?(?:\s(\d+(?:\.\d+)?)\s?pence)?'
    
    # Try to find a coherent group first
    group_match = re.search(monetary_group_pattern, sentence)
    if group_match:
        monetary_amounts['Pounds'] = float(group_match.group(1) or 0)
        monetary_amounts['Shillings'] = float(group_match.group(2) or 0)
        monetary_amounts['Pence'] = float(group_match.group(3) or 0)
        return monetary_amounts

    # If no coherent group is found, fallback to individual searches (This part is more of a safety net)
    pounds_found = re.search(r'£(\d+(?:\.\d+)?)', sentence)
    shillings_found = re.search(r'(\d+(?:\.\d+)?)\s?shilling', sentence)
    pence_found = re.search(r'(\d+(?:\.\d+)?)\s?pence', sentence)

    if pounds_found:
        monetary_amounts['Pounds'] = float(pounds_found.group(1))
    if shillings_found:
        monetary_amounts['Shillings'] = float(shillings_found.group(1))
    if pence_found:
        monetary_amounts['Pence'] = float(pence_found.group(1))

    return monetary_amounts

document_names = [
    "Adderbury", "Alresford", "AlresfordBorough", "Alverstoke", "Ashmansworth",
    "Beauworth", "Bentley", "BishopsFonthill", "BishopsSutton", "BishopsWaltham",
    "Bitterne", "Brightwell", "Burghclere", "Cams", "Cheriton", "Crawley", "Culham",
    "Downton", "DowntonBorough", "Droxford", "EastKnoyle", "EastMeon", "EastMeonChurch",
    "Ecchinswell", "Esher", "Farnham", "Gosport", "Hambledon", "Harwell", "Havant",
    "Highclere", "HindonBorough", "Holway", "Ivinghoe", "Kingston", "Marshalsea",
    "Merdon", "Morton", "Newtown", "NorthWaltham", "Otterford", "Overton",
    "OvertonBorough", "Poundisford", "Rimpton", "SokeofWinchester", "Southwark",
    "Staplegrove", "StGilesFair", "Taunton", "TauntonBorough", "TheChurch",
    "TrullBishopsHull", "Twyford", "Upton", "WalthamStLawrence", "Warfield", "Wargrave",
    "Warren", "WestWycombe", "Wield", "Witney", "WitneyBorough", "Wolvesey", "Woodhay"
]

# Base paths for the documents and Excel files
doc_base_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\input\OCR 1301\\"
excel_base_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script1\\"

for doc_name in document_names:
    # Construct the full paths for the input and output files
    doc_path = f'{doc_base_path}{doc_name}.docx'
    excel_path = f'{excel_base_path}{doc_name}1.xlsx'

    # Load the Word Document
    doc = docx.Document(doc_path)

# Load the Word Document (old versions, now a loop through all documents right above)
#doc_path = '/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301clean.docx'
#doc_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/OCR 1301/Taunton.docx'
#doc = docx.Document(doc_path)

    data = []
    current_heading = None

    #Split into columns and calculate total monetary amount
    for para in doc.paragraphs:
        if is_heading(para):
            current_heading = para.text
        else:
            if current_heading:
                sentences = split_into_sentences(para.text)
                for sentence in sentences:
                    if sentence.strip():
                        preprocessed_sentence = sentence.replace('1/2', '.5').replace('1/4', '.25')
    
                        monetary_amounts = extract_first_monetary_amount(preprocessed_sentence)
    
                        total_pounds = float(monetary_amounts['Pounds']) + float(monetary_amounts['Shillings']) / 20 + float(monetary_amounts['Pence']) / 240
                        total_pounds = round(total_pounds, 2)
    
                        data.append({
                            "Heading": current_heading, 
                            "Sentence": preprocessed_sentence,
                            "Total in Pounds": total_pounds, 
                            **monetary_amounts
                        })


    # Create a DataFrame and write to Excel
    df = pd.DataFrame(data)

    # Placeholder DataFrames for additional sheets
    # Replace these with your actual data as needed
    overview_df = pd.DataFrame()  # and so on for other DataFrames...
    
    # Dictionary of DataFrames for each sheet
    dfs = {
        'raw data': df,
        'Overview': overview_df,
        'Receipts': pd.DataFrame(),  # Replace with actual data
        'Expenses': pd.DataFrame(),
        'Issues of the Grange': pd.DataFrame(),
        'Issues of the Mills': pd.DataFrame(),
        'Stock': pd.DataFrame(),
        'Prices': pd.DataFrame(),
        'Labour Rents': pd.DataFrame(),
        'not needed' : pd.DataFrame()
    }

# File path for the Excel file
#excel_path = '/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301.xlsx'
#excel_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script1/Taunton1.xlsx'

    # Using ExcelWriter to write multiple DataFrames to different sheets
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for sheet_name, data in dfs.items():
            data.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"Excel file for {doc_name} created successfully.")




Excel file for Adderbury created successfully.
Excel file for Alresford created successfully.
Excel file for AlresfordBorough created successfully.
Excel file for Alverstoke created successfully.
Excel file for Ashmansworth created successfully.
Excel file for Beauworth created successfully.
Excel file for Bentley created successfully.
Excel file for BishopsFonthill created successfully.
Excel file for BishopsSutton created successfully.
Excel file for BishopsWaltham created successfully.
Excel file for Bitterne created successfully.
Excel file for Brightwell created successfully.
Excel file for Burghclere created successfully.
Excel file for Cams created successfully.
Excel file for Cheriton created successfully.
Excel file for Crawley created successfully.
Excel file for Culham created successfully.
Excel file for Downton created successfully.
Excel file for DowntonBorough created successfully.
Excel file for Droxford created successfully.
Excel file for EastKnoyle created successful

In [9]:
####################### SCRIPT 2  ####################### 

#This script will sort the different payments into receipts, expenditure, etc.

import pandas as pd
import openpyxl

document_names = [
    "Adderbury", "Alresford", "AlresfordBorough", "Alverstoke", "Ashmansworth",
    "Beauworth", "Bentley", "BishopsFonthill", "BishopsSutton", "BishopsWaltham",
    "Bitterne", "Brightwell", "Burghclere", "Cams", "Cheriton", "Crawley", "Culham",
    "Downton", "DowntonBorough", "Droxford", "EastKnoyle", "EastMeon", "EastMeonChurch",
    "Ecchinswell", "Esher", "Farnham", "Gosport", "Hambledon", "Harwell", "Havant",
    "Highclere", "HindonBorough", "Holway", "Ivinghoe", "Kingston", "Marshalsea",
    "Merdon", "Morton", "Newtown", "NorthWaltham", "Otterford", "Overton",
    "OvertonBorough", "Poundisford", "Rimpton", "SokeofWinchester", "Southwark",
    "Staplegrove", "StGilesFair", "Taunton", "TauntonBorough", "TheChurch",
    "TrullBishopsHull", "Twyford", "Upton", "WalthamStLawrence", "Warfield", "Wargrave",
    "Warren", "WestWycombe", "Wield", "Witney", "WitneyBorough", "Wolvesey", "Woodhay"
]


# Specify the paths to your Excel files
type_excel_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\input\Type.xlsx"

# The loop will dynamically create these paths for each document
input_base_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script1\\"
output_base_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script2\\"


# Load the 'Type' Excel file
type_df = pd.read_excel(type_excel_path)

# Create a dictionary to map headings to their types
type_mapping = dict(zip(type_df['Heading'], type_df['Type']))

#new code
for doc_name in document_names:
    # Dynamically construct the specific paths for each document
    manorial_excel_path = f'{input_base_path}{doc_name}1.xlsx'
    new_excel_path = f'{output_base_path}{doc_name}2.xlsx'

    # Load the 'raw data' sheet and the entire workbook for each document
    raw_data_df = pd.read_excel(manorial_excel_path, sheet_name='raw data')
    book = openpyxl.load_workbook(manorial_excel_path)


    # Define the column headers
    column_headers = ['Heading', 'Sentence', 'Total in Pounds', 'Pounds', 'Shillings', 'Pence']
    
    # Dictionary to store the data for each sheet, with totals calculation
    sheets_data = {}
    totals = {}  # Dictionary to store totals for each heading
    
    
    # Iterate through each row in the 'raw data' DataFrame
    last_heading = None
    exclude_keywords = ['Total', 'total', 'In total']  # Keywords to exclude in total calculation

    for index, row in raw_data_df.iterrows():
        heading = row['Heading']
        if heading in type_mapping:
            target_sheet_name = type_mapping.get(heading)
            if target_sheet_name:
                if target_sheet_name not in sheets_data:
                    sheets_data[target_sheet_name] = []
    
                # Add an empty row before a new heading
                if last_heading != heading:
                    if last_heading is not None:
                        # Append total for the last heading
                        total_row = {"Heading": last_heading, "Sentence": "Total", "Total in Pounds": totals.get(last_heading, 0)}
                        sheets_data[target_sheet_name].append(total_row)
                        # Append an empty row
                        sheets_data[target_sheet_name].append({})
    
                    last_heading = heading
                    totals[heading] = 0  # Reset total for new heading
                    # Append an empty row
                    sheets_data[target_sheet_name].append({})
    
                # Check if the sentence starts with any of the excluded keywords
                if not any(row['Sentence'].startswith(kw) for kw in exclude_keywords):
                    # Update the total for the current heading
                    totals[heading] += float(row['Total in Pounds'])
    
                # Process and append the current row
                sheets_data[target_sheet_name].append(row)
    
    # Add the total for the last heading in the loop
    if last_heading:
        total_row = {"Heading": last_heading, "Sentence": "Total", "Total in Pounds": totals.get(last_heading, 0)}
        sheets_data[target_sheet_name].append(total_row)


    # Write the aggregated data to the respective sheets in 'Manorial1301.xlsx'
    for sheet_name, data in sheets_data.items():
        sheet_df = pd.DataFrame(data)
    
        # Ensure the workbook has the sheet
        if sheet_name not in book.sheetnames:
            book.create_sheet(sheet_name)
            target_sheet = book[sheet_name]
            target_sheet.append(column_headers)  # Add column headers for a new sheet
        else:
            target_sheet = book[sheet_name]
            if target_sheet.max_row == 1:
                target_sheet.append(column_headers)  # Add column headers if the sheet is empty
    
        # Write data to the sheet
        for index, row in sheet_df.iterrows():
            # Check if the row is not empty (to skip empty rows added for spacing)
            if not row.isnull().all():
                target_sheet.append(row.tolist())


    # After processing, save the workbook to the new path for each document
    book.save(new_excel_path)
    print(f"Processed and saved: {new_excel_path}")


Processed and saved: C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script2\\Adderbury2.xlsx
Processed and saved: C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script2\\Alresford2.xlsx
Processed and saved: C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script2\\AlresfordBorough2.xlsx
Processed and saved: C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script2\\Alverstoke2.xlsx
Processed and saved: C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script2\\Ashmansworth2.xlsx
Processed and saved

In [35]:
#This works 

####################### SCRIPT 3  ####################### 

################## Creating Error Column ################## 
                        # & #
######### Separating out Bushels and Quarters #############
  
#extract pairs of shil-penc per quart-bush

import pandas as pd
import re
import os 

# Function to ensure the directory exists for the output path
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# List of document names
document_names = [
    "Adderbury", "Alresford", "AlresfordBorough", "Alverstoke", "Ashmansworth",
    "Beauworth", "Bentley", "BishopsFonthill", "BishopsSutton", "BishopsWaltham",
    "Bitterne", "Brightwell", "Burghclere", "Cams", "Cheriton", "Crawley", "Culham",
    "Downton", "DowntonBorough", "Droxford", "EastKnoyle", "EastMeon", "EastMeonChurch",
    "Ecchinswell", "Esher", "Farnham", "Gosport", "Hambledon", "Harwell", "Havant",
    "Highclere", "HindonBorough", "Holway", "Ivinghoe", "Kingston", "Marshalsea",
    "Merdon", "Morton", "Newtown", "NorthWaltham", "Otterford", "Overton",
    "OvertonBorough", "Poundisford", "Rimpton", "SokeofWinchester", "Southwark",
    "Staplegrove", "StGilesFair", "Taunton", "TauntonBorough", "TheChurch",
    "TrullBishopsHull", "Twyford", "Upton", "WalthamStLawrence", "Warfield", "Wargrave",
    "Warren", "WestWycombe", "Wield", "Witney", "WitneyBorough", "Wolvesey", "Woodhay"
]

# List of all required sheet names in the desired order
required_sheets = [
    "raw data", "Overview", "Receipts", "Expenses", 
    "Issues of the Grange", "Issues of the Mills", 
    "Stock", "Prices", "Labour Rents", "not needed"
]

# Define base paths
input_base_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script2\\"
output_base_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script3\\"




#1) Replace abbrevations qrs. for quarters
def replace_abbreviations(sentence):
    sentence = re.sub(r'\bqr\.\b', 'quarters', sentence)
    sentence = re.sub(r'\bqr\b', 'quarters', sentence)
    sentence = re.sub(r'\bbus\b', 'bushels', sentence)  # Add this line
    return sentence

#2) Parse out the type of corn mentioned in each sentence
def extract_corn_type(sentence):
    # List of specific corn types to look for
    corn_types = ['wheat', 'mancorn', 'maslin', 'barley', 'first-grade malt', 'second-grade malt', 'peas', 'oats', 'oatmeal', 'vetches', 'rye', 'curall', 'beans', 'dredge', 'malt', 'meal', 'curall']
    
    # Find corn types in the sentence
    found_corn_types = []
    for corn in corn_types:
        # Use case-insensitive search for each corn type
        if re.search(re.escape(corn), sentence, re.IGNORECASE):
            found_corn_types.append(corn)

    # Return the unique corn types found in the sentence
    return ', '.join(set(found_corn_types))  # Join multiple types with a comma if needed

#3) Extract quantities of quarters and bushels  
def extract_quantities(sentence):
    # Updated pattern to capture pairs of quarters and bushels
    pattern = r'(\d+(?:\.\d+)?)\s?(quarters|bushels)(?:\s?(\d+(?:\.\d+)?)\s?(quarters|bushels))?'
    matches = re.findall(pattern, sentence)

    quantities = []
    for match in matches:
        # Initialize default values for quarters and bushels
        quarters = 0
        bushels = 0

        # Assign values based on the matched groups
        if match[1] == 'quarters':
            quarters = float(match[0])
            if match[3] == 'bushels':
                bushels = float(match[2]) if match[2] else 0
        elif match[1] == 'bushels':
            bushels = float(match[0])
            if match[3] == 'quarters':
                quarters = float(match[2]) if match[2] else 0

        # Append the pair to the quantities list
        quantities.append({'Quarters': quarters, 'Bushels': bushels})

    return quantities

# Example usage
sentence = "£8 10 shilling 6.5pence from 29.5 quarters of wheat sold, at 4 shilling a qr for 5.5quarters, 5 shilling a qr for 2 quarters 5 bushels, 6 shilling a qr for 10 quarters 5 bushels, and, 6 shilling 8pence a qr for 10 quarters 6 bushels."
quantities = extract_quantities(sentence)

#4) Insert empty rows for legibility and calculate error term 
def insert_empty_rows_and_calculate_error(df):
    # Insert an empty row after the first row
    df = pd.concat([df.iloc[:1], pd.DataFrame([['']*len(df.columns)], columns=df.columns), df.iloc[1:]]).reset_index(drop=True)

    # Add a placeholder for the new 'Error' column
    df['Error'] = None

    # Group by 'Heading' and process each group
    grouped = df.groupby('Heading', sort=False)
    new_df = pd.DataFrame()

    for _, group in grouped:
        if len(group) > 1:
            # Calculate the Error for the group (last value - penultimate value in 'Total in Pounds')
            last_value = group['Total in Pounds'].iloc[-1]
            penultimate_value = group['Total in Pounds'].iloc[-2]
            error = last_value - penultimate_value
            # Assign the Error value to the last row of the group
            group.at[group.index[-1], 'Error'] = error

        # Append two empty rows and the group to the new dataframe
        group_with_empty_rows = pd.concat([group, pd.DataFrame([['']*len(df.columns)]*2, columns=df.columns)], ignore_index=True)
        new_df = pd.concat([new_df, group_with_empty_rows], ignore_index=True)

    return new_df

#5) Extracting monetary amounts of per quarter-bushel sales
def extract_monetary_amounts(sentence):
    patterns = {
        'shilling and pence a quarter': r'(\d+(?:\.\d+)?)\s?shilling(?:s)?(?:\s?(\d+(?:\.\d+)?)\s?pence)?\s?a\s?quarter(?:s)?',
        'shilling and pence a bushel': r'((?:\d+(?:\.\d+)?)\s?shilling(?:s)?\s?)?(?:\s?(\d+(?:\.\d+)?)\s?pence)?\s?a\s?bushel(?:s)?'
    }

    # Initialize a dictionary to store results
    results = {key: [] for key in patterns}

    # Iterate through each pattern and find all matches in the sentence
    for key, pattern in patterns.items():
        matches = re.findall(pattern, sentence, re.IGNORECASE)
        for match in matches:
            shilling = float(match[0]) if match[0] else 0
            pence = float(match[1]) if match[1] else 0
            results[key].append((shilling, pence))
            
    return results

# Example sentences
example_sentences = [
    "The same render account for £8 10 shilling 6.5pence from 29.5 quarters of wheat sold from the issue of the same mill, at 4 shilling a quarters for 5.5quarters, 5 shilling a quarters for 2 quarters 5 bushels, 6 shilling a quarters for 10 quarters 5 bushels, and, 6 shilling 8pence a quarters for 10 quarters 6 bushels.",
    # Add more sentences here for testing
]

# Test the function
#for sentence in example_sentences:
#    extracted_amounts = extract_monetary_amounts(sentence)
#    print(f"Sentence: {sentence}\nExtracted Amounts: {extracted_amounts}\n")


#old input paths
#xls = pd.ExcelFile('/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_updated.xlsx')
#xls = pd.ExcelFile('/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script2/Taunton2.xlsx')


#old output paths
#create new file path to which the output below is saved
#new_file_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script3/Taunton3.xlsx'



# Process each document
for doc_name in document_names:
    input_path = f'{input_base_path}{doc_name}2.xlsx'
    output_path = f'{output_base_path}{doc_name}3.xlsx'
    ensure_dir(output_path)

    # Check if the input file exists to process
    if not os.path.exists(input_path):
        print(f"File does not exist: {input_path}")
        continue

    # Load the Excel file
    xls = pd.ExcelFile(input_path)
    
    # Process each sheet
    #In the final part of your script where you use pd.ExcelWriter to save processed data, 
    #the variable new_file_path is meant to specify the filename and path where the changes should be saved. 
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name)
            if 'Heading' not in df.columns:
                print(f"Skipping '{sheet_name}' in {doc_name} as it lacks 'Heading' column.")
                continue  # Skip this sheet processing

            
            # Initialize necessary columns to ensure they exist
            for col_name in ['Quarters 1', 'Bushels 1']:
                if col_name not in df.columns:
                    df[col_name] = 0.0  # Initialize with default value


            if sheet_name == 'raw data':
                # Initialize columns for monetary amounts
                monetary_cols = ['shilling a quarter', 'pence a quarter', 'shilling a bushel', 'pence a bushel']
                for col in monetary_cols:
                    for i in range(1, 5):  # Adjust range based on the expected max count
                        df[f'{col} {i}'] = None
    
                for index, row in df.iterrows():
                    # Update 'Sentence' with replaced abbreviations
                    updated_sentence = replace_abbreviations(row['Sentence'])
                    df.at[index, 'Sentence'] = updated_sentence
    
                    # Extract monetary amounts from the updated sentence
                    monetary_amounts = extract_monetary_amounts(updated_sentence)
    
                    # This is the correct handling of the monetary amounts extraction
                    for i, (shilling, pence) in enumerate(monetary_amounts['shilling and pence a quarter']):
                        df.at[index, f'shilling a quarter {i + 1}'] = shilling
                        df.at[index, f'pence a quarter {i + 1}'] = pence
    
                    for i, (shilling, pence) in enumerate(monetary_amounts['shilling and pence a bushel']):
                        df.at[index, f'shilling a bushel {i + 1}'] = shilling
                        df.at[index, f'pence a bushel {i + 1}'] = pence
    
                    #print(f"Processing Updated Sentence: {updated_sentence}")  # Debugging print
                    monetary_amounts = extract_monetary_amounts(updated_sentence)
                    #print("Monetary Amounts:", monetary_amounts)  # Debugging print

            #print(df[['shilling a quarter 1', 'shilling a quarter 2', 'shilling a bushel 1', 'shilling a bushel 2']].head())  # Adjust column names as per your DataFrame


            #for col in monetary_cols:
            #   for i, value in enumerate(monetary_amounts[col]):
            #        col_name = f'{col} {i + 1}'
            #       print(f"Assigning {value} to column {col_name}")  # Debugging print
            #       if col_name in df.columns:
            #           df.at[index, col_name] = value
      
                # Add new columns for 'Corn', 'Quarters', and 'Bushels'
                df['Corn'] = None
                max_pairs = 0
                for index, row in df.iterrows():
                    # Update 'Sentence' with replaced abbreviations
                    updated_sentence = replace_abbreviations(row['Sentence'])
                    df.at[index, 'Sentence'] = updated_sentence
                    
                    # Extract and update 'Corn' types
                    corn_type = extract_corn_type(updated_sentence)
                    df.at[index, 'Corn'] = corn_type
    
                    # Extract quantities and update 'Quarters' and 'Bushels'
                    quantities = extract_quantities(updated_sentence)
                    max_pairs = max(max_pairs, len(quantities))
                    for i, quantity_pair in enumerate(quantities):
                        df.at[index, f'Quarters {i+1}'] = quantity_pair['Quarters']
                        df.at[index, f'Bushels {i+1}'] = quantity_pair['Bushels']
    
                # Add alternating columns for quarters and bushels with float dtype
                for i in range(max_pairs):
                    if f'Quarters {i+1}' not in df.columns:
                        df[f'Quarters {i+1}'] = pd.Series(dtype='float')
                    if f'Bushels {i+1}' not in df.columns:
                        df[f'Bushels {i+1}'] = pd.Series(dtype='float')

                # Initialize new columns for total quarters and bushels
                df['Total Quarters Except 1'] = 0.0
                df['Total Bushels Except 1'] = 0.0
                df['Total Corn Cross-Check'] = 0.0
                df['Total Corn Manorial Account'] = 0.0
                df['Corn Quantity Error'] = 0.0
    
                for index, row in df.iterrows():
                    total_quarters = 0.0
                    total_bushels = 0.0
    
                   # Sum up quarters and bushels from 2nd column onwards
                    for i in range(2, max_pairs + 1):
                        quarters_col = f'Quarters {i}'
                        bushels_col = f'Bushels {i}'
    
                       # Check if the column exists and add its value if it does
                        if quarters_col in df.columns:
                            total_quarters += row[quarters_col] if not pd.isna(row[quarters_col]) else 0.0
                        if bushels_col in df.columns:
                            total_bushels += row[bushels_col] if not pd.isna(row[bushels_col]) else 0.0
    
                    # Assign Quarter 1 and Bushel 1 to totals if no other quarters and bushels exist
                    #if total_quarters == 0 and total_bushels == 0 and 'Quarters 1' in df.columns and 'Bushels 1' in df.columns:
                    #    total_quarters = row['Quarters 1'] if not pd.isna(row['Quarters 1']) else 0.0
                    #    total_bushels = row['Bushels 1'] if not pd.isna(row['Bushels 1']) else 0.0


                    # Assign Quarter 1 and Bushel 1 to totals if no other quarters and bushels exist
                    if 'Quarters 1' in df.columns and 'Bushels 1' in df.columns:
                        if total_quarters == 0:
                            total_quarters = row['Quarters 1'] if not pd.isna(row['Quarters 1']) else 0.0
                        if total_bushels == 0:
                            total_bushels = row['Bushels 1'] if not pd.isna(row['Bushels 1']) else 0.0


                    df.at[index, 'Total Quarters Except 1'] = total_quarters
                    df.at[index, 'Total Bushels Except 1'] = total_bushels
                    df.at[index, 'Total Corn Cross-Check'] = total_quarters + (total_bushels / 8)
                    df.at[index, 'Total Corn Manorial Account'] = (row['Quarters 1'] if 'Quarters 1' in df.columns and not pd.isna(row['Quarters 1']) else 0) + \
                                                                  ((row['Bushels 1'] if 'Bushels 1' in df.columns and not pd.isna(row['Bushels 1']) else 0) / 8)
                    df.at[index, 'Corn Quantity Error'] = df.at[index, 'Total Corn Cross-Check'] - df.at[index, 'Total Corn Manorial Account']

                    
                    # Store the sums in the new columns
                    #df.at[index, 'Total Quarters Except 1'] = total_quarters
                    #df.at[index, 'Total Bushels Except 1'] = total_bushels

                   # Calculate Total Corn Cross-Check and Total Corn Manorial Account
                   # df.at[index, 'Total Corn Cross-Check'] = total_quarters + (total_bushels / 8)
                   # quarters_1 = row['Quarters 1'] if not pd.isna(row['Quarters 1']) else 0.0
                   # bushels_1 = row['Bushels 1'] if not pd.isna(row['Bushels 1']) else 0.0
                   # df.at[index, 'Total Corn Manorial Account'] = quarters_1 + (bushels_1 / 8)
    
                    # Calculate Corn Quantity Error
                    cross_check_total = df.at[index, 'Total Corn Cross-Check']
                    manorial_account_total = df.at[index, 'Total Corn Manorial Account']
                    df.at[index, 'Corn Quantity Error'] = cross_check_total - manorial_account_total
    
    
                # Define the initial set of columns
                ordered_columns = ['Heading', 'Sentence', 'Total in Pounds', 'Pounds', 'Shillings', 'Pence', 'Corn', 
                       'Total Quarters Except 1', 'Total Bushels Except 1', 'Total Corn Cross-Check', 
                       'Total Corn Manorial Account', 'Corn Quantity Error']


                # Determine the number of 'Quarters' and 'Bushels' columns
                num_quarters_bushels_columns = max([int(col.split(' ')[-1]) for col in df.columns if 'Quarters' in col or 'Bushels' in col], default=0)
    
                # Add 'Quarters' and 'Bushels' columns in pairs
                for i in range(1, num_quarters_bushels_columns + 1):
                    ordered_columns.append(f'Quarters {i}')
                    ordered_columns.append(f'Bushels {i}')
    
                # Determine the number of 'shilling a quarter' and 'pence a quarter' columns
                num_quarter_columns = max([int(col.split(' ')[-1]) for col in df.columns if 'shilling a quarter' in col or 'pence a quarter' in col], default=0)
    
                # Add 'shilling a quarter' and 'pence a quarter' columns in pairs
                for i in range(1, num_quarter_columns + 1):
                    ordered_columns.append(f'shilling a quarter {i}')
                    ordered_columns.append(f'pence a quarter {i}')
    
                # Determine the number of 'shilling a bushel' and 'pence a bushel' columns
                num_bushel_columns = max([int(col.split(' ')[-1]) for col in df.columns if 'shilling a bushel' in col or 'pence a bushel' in col], default=0)
    
                # Add 'shilling a bushel' and 'pence a bushel' columns in pairs
                for i in range(1, num_bushel_columns + 1):
                    ordered_columns.append(f'shilling a bushel {i}')
                    ordered_columns.append(f'pence a bushel {i}')
    
                # Reorder the DataFrame
                df = df[ordered_columns]


            elif sheet_name in ['Receipts', 'Expenses']:
                # Process 'Receipts' and 'Expenses' sheets with your error calculation
                df = insert_empty_rows_and_calculate_error(df)


            # Example of safely accessing the columns
            if 'Quarters 1' in df.columns and 'Bushels 1' in df.columns:
            # Perform operations with these columns
                pass

            # Save each processed sheet to the new Excel file
            df.to_excel(writer, sheet_name=sheet_name, index=False)


        # At the end of the Excel writing block (right before closing the writer context)
        for sheet_name in required_sheets:
            if sheet_name not in writer.book.sheetnames:
                # Create an empty DataFrame
                empty_df = pd.DataFrame()
                # Save it as an empty sheet
                empty_df.to_excel(writer, sheet_name=sheet_name, index=False)

         # Ensure there is an active sheet at the end of processing
        if writer.book.sheetnames:
            # Set the first sheet in the workbook as active
            first_sheet = writer.book[writer.book.sheetnames[0]]
            writer.book.active = first_sheet
    
        print(f"Processed and saved: {output_path}")






Skipping 'Overview' in Adderbury as it lacks 'Heading' column.
Skipping 'Issues of the Mills' in Adderbury as it lacks 'Heading' column.
Skipping 'Prices' in Adderbury as it lacks 'Heading' column.
Skipping 'Labour Rents' in Adderbury as it lacks 'Heading' column.
Skipping 'not needed' in Adderbury as it lacks 'Heading' column.
Processed and saved: C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script3\\Adderbury3.xlsx
Skipping 'Overview' in Alresford as it lacks 'Heading' column.
Skipping 'Issues of the Mills' in Alresford as it lacks 'Heading' column.
Skipping 'Prices' in Alresford as it lacks 'Heading' column.
Skipping 'Labour Rents' in Alresford as it lacks 'Heading' column.
Skipping 'not needed' in Alresford as it lacks 'Heading' column.
Processed and saved: C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-130

In [ ]:
####################### SCRIPT 3  #######################  

 

################## Creating Error Column ##################  

                        # & # 

######### Separating out Bushels and Quarters ############# 

   

#extract pairs of shil-penc per quart-bush 

 

import pandas as pd 

import re 

 

# Define your functions here 

#1) Replace abbrevations qrs. for quarters 

def replace_abbreviations(sentence): 

    sentence = re.sub(r'\bqr\.\b', 'quarters', sentence) 

    sentence = re.sub(r'\bqr\b', 'quarters', sentence) 

    sentence = re.sub(r'\bbus\b', 'bushels', sentence)  # Add this line 

    return sentence 

 

#2) Parse out the type of corn mentioned in each sentence 

def extract_corn_type(sentence): 

    # List of specific corn types to look for 

    corn_types = ['wheat', 'maslin', 'barley', 'first-grade malt', 'second-grade malt', 'peas', 'oats', 'oatmeal', 'vetches', 'rye', 'curall', 'beans'] 

     

    # Find corn types in the sentence 

    found_corn_types = [] 

    for corn in corn_types: 

        # Use case-insensitive search for each corn type 

        if re.search(re.escape(corn), sentence, re.IGNORECASE): 

            found_corn_types.append(corn) 

 

    # Return the unique corn types found in the sentence 

    return ', '.join(set(found_corn_types))  # Join multiple types with a comma if needed 

 

#3) Extract quantities of quarters and bushels   

def extract_quantities(sentence): 

    # Updated pattern to capture pairs of quarters and bushels 

    pattern = r'(\d+(?:\.\d+)?)\s?(quarters|bushels)(?:\s?(\d+(?:\.\d+)?)\s?(quarters|bushels))?' 

    matches = re.findall(pattern, sentence) 

 

    quantities = [] 

    for match in matches: 

        # Initialize default values for quarters and bushels 

        quarters = 0 

        bushels = 0 

 

        # Assign values based on the matched groups 

        if match[1] == 'quarters': 

            quarters = float(match[0]) 

            if match[3] == 'bushels': 

                bushels = float(match[2]) if match[2] else 0 

        elif match[1] == 'bushels': 

            bushels = float(match[0]) 

            if match[3] == 'quarters': 

                quarters = float(match[2]) if match[2] else 0 

 

        # Append the pair to the quantities list 

        quantities.append({'Quarters': quarters, 'Bushels': bushels}) 

 

    return quantities 

 

# Example usage 

sentence = "£8 10 shilling 6.5pence from 29.5 quarters of wheat sold, at 4 shilling a qr for 5.5quarters, 5 shilling a qr for 2 quarters 5 bushels, 6 shilling a qr for 10 quarters 5 bushels, and, 6 shilling 8pence a qr for 10 quarters 6 bushels." 

quantities = extract_quantities(sentence) 

 

#4) Insert empty rows for legibility and calculate error term  

def insert_empty_rows_and_calculate_error(df): 

    # Insert an empty row after the first row 

    df = pd.concat([df.iloc[:1], pd.DataFrame([['']*len(df.columns)], columns=df.columns), df.iloc[1:]]).reset_index(drop=True) 

 

    # Add a placeholder for the new 'Error' column 

    df['Error'] = None 

 

    # Group by 'Heading' and process each group 

    grouped = df.groupby('Heading', sort=False) 

    new_df = pd.DataFrame() 

 

    for _, group in grouped: 

        if len(group) > 1: 

            # Calculate the Error for the group (last value - penultimate value in 'Total in Pounds') 

            last_value = group['Total in Pounds'].iloc[-1] 

            penultimate_value = group['Total in Pounds'].iloc[-2] 

            error = last_value - penultimate_value 

            # Assign the Error value to the last row of the group 

            group.at[group.index[-1], 'Error'] = error 

 

        # Append two empty rows and the group to the new dataframe 

        group_with_empty_rows = pd.concat([group, pd.DataFrame([['']*len(df.columns)]*2, columns=df.columns)], ignore_index=True) 

        new_df = pd.concat([new_df, group_with_empty_rows], ignore_index=True) 

 

    return new_df 

 

#5) Extracting monetary amounts of per quarter-bushel sales 

def extract_monetary_amounts(sentence): 

    patterns = { 

        'shilling and pence a quarter': r'(\d+(?:\.\d+)?)\s?shilling(?:s)?(?:\s?(\d+(?:\.\d+)?)\s?pence)?\s?a\s?quarter(?:s)?', 

        'shilling and pence a bushel': r'((?:\d+(?:\.\d+)?)\s?shilling(?:s)?\s?)?(?:\s?(\d+(?:\.\d+)?)\s?pence)?\s?a\s?bushel(?:s)?' 

    } 

 

    # Initialize a dictionary to store results 

    results = {key: [] for key in patterns} 

 

    # Iterate through each pattern and find all matches in the sentence 

    for key, pattern in patterns.items(): 

        matches = re.findall(pattern, sentence, re.IGNORECASE) 

        for match in matches: 

            shilling = float(match[0]) if match[0] else 0 

            pence = float(match[1]) if match[1] else 0 

            results[key].append((shilling, pence)) 

             

    return results 

 

# Example sentences 

example_sentences = [ 

    "The same render account for £8 10 shilling 6.5pence from 29.5 quarters of wheat sold from the issue of the same mill, at 4 shilling a quarters for 5.5quarters, 5 shilling a quarters for 2 quarters 5 bushels, 6 shilling a quarters for 10 quarters 5 bushels, and, 6 shilling 8pence a quarters for 10 quarters 6 bushels.", 

    # Add more sentences here for testing 

] 

 

# Test the function 

#for sentence in example_sentences: 

#    extracted_amounts = extract_monetary_amounts(sentence) 

#    print(f"Sentence: {sentence}\nExtracted Amounts: {extracted_amounts}\n") 

 

 

# Now, use your for loop where you call these functions 

xls = pd.ExcelFile('/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_updated.xlsx') 

 

#create new file path to which the output below is saved 

new_file_path = '/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_sorted.xlsx' 

 

 

# Process each sheet 

#In the final part of your script where you use pd.ExcelWriter to save processed data,  

#the variable new_file_path is meant to specify the filename and path where the changes should be saved.  

with pd.ExcelWriter(new_file_path, engine='openpyxl') as writer: 

    for sheet_name in xls.sheet_names: 

        df = pd.read_excel(xls, sheet_name) 

 

        if sheet_name == 'raw data': 

            # Initialize columns for monetary amounts 

            monetary_cols = ['shilling a quarter', 'pence a quarter', 'shilling a bushel', 'pence a bushel'] 

            for col in monetary_cols: 

                for i in range(1, 5):  # Adjust range based on the expected max count 

                    df[f'{col} {i}'] = None 

 

            for index, row in df.iterrows(): 

                # Update 'Sentence' with replaced abbreviations 

                updated_sentence = replace_abbreviations(row['Sentence']) 

                df.at[index, 'Sentence'] = updated_sentence 

 

                # Extract monetary amounts from the updated sentence 

                monetary_amounts = extract_monetary_amounts(updated_sentence) 

 

                # This is the correct handling of the monetary amounts extraction 

                for i, (shilling, pence) in enumerate(monetary_amounts['shilling and pence a quarter']): 

                    df.at[index, f'shilling a quarter {i + 1}'] = shilling 

                    df.at[index, f'pence a quarter {i + 1}'] = pence 

 

                for i, (shilling, pence) in enumerate(monetary_amounts['shilling and pence a bushel']): 

                    df.at[index, f'shilling a bushel {i + 1}'] = shilling 

                    df.at[index, f'pence a bushel {i + 1}'] = pence 

 

                #print(f"Processing Updated Sentence: {updated_sentence}")  # Debugging print 

                monetary_amounts = extract_monetary_amounts(updated_sentence) 

                #print("Monetary Amounts:", monetary_amounts)  # Debugging print 

 

            #print(df[['shilling a quarter 1', 'shilling a quarter 2', 'shilling a bushel 1', 'shilling a bushel 2']].head())  # Adjust column names as per your DataFrame 

 

 

            #for col in monetary_cols: 

            #   for i, value in enumerate(monetary_amounts[col]): 

            #        col_name = f'{col} {i + 1}' 

            #       print(f"Assigning {value} to column {col_name}")  # Debugging print 

            #       if col_name in df.columns: 

            #           df.at[index, col_name] = value 

       

            # Add new columns for 'Corn', 'Quarters', and 'Bushels' 

            df['Corn'] = None 

            max_pairs = 0 

            for index, row in df.iterrows(): 

                # Update 'Sentence' with replaced abbreviations 

                updated_sentence = replace_abbreviations(row['Sentence']) 

                df.at[index, 'Sentence'] = updated_sentence 

                 

                # Extract and update 'Corn' types 

                corn_type = extract_corn_type(updated_sentence) 

                df.at[index, 'Corn'] = corn_type 

 

                # Extract quantities and update 'Quarters' and 'Bushels' 

                quantities = extract_quantities(updated_sentence) 

                max_pairs = max(max_pairs, len(quantities)) 

                for i, quantity_pair in enumerate(quantities): 

                    df.at[index, f'Quarters {i+1}'] = quantity_pair['Quarters'] 

                    df.at[index, f'Bushels {i+1}'] = quantity_pair['Bushels'] 

 

            # Add alternating columns for quarters and bushels with float dtype 

            for i in range(max_pairs): 

                if f'Quarters {i+1}' not in df.columns: 

                    df[f'Quarters {i+1}'] = pd.Series(dtype='float') 

                if f'Bushels {i+1}' not in df.columns: 

                    df[f'Bushels {i+1}'] = pd.Series(dtype='float') 

 

            # Initialize new columns for total quarters and bushels 

            df['Total Quarters Except 1'] = 0.0 

            df['Total Bushels Except 1'] = 0.0 

            df['Total Corn Cross-Check'] = 0.0 

            df['Total Corn Manorial Account'] = 0.0 

            df['Corn Quantity Error'] = 0.0 

 

            for index, row in df.iterrows(): 

                total_quarters = 0.0 

                total_bushels = 0.0 

 

               # Sum up quarters and bushels from 2nd column onwards 

                for i in range(2, max_pairs + 1): 

                    quarters_col = f'Quarters {i}' 

                    bushels_col = f'Bushels {i}' 

 

               # Check if the column exists and add its value if it does 

                    if quarters_col in df.columns: 

                        total_quarters += row[quarters_col] if not pd.isna(row[quarters_col]) else 0.0 

                    if bushels_col in df.columns: 

                        total_bushels += row[bushels_col] if not pd.isna(row[bushels_col]) else 0.0 

 

                # Assign Quarter 1 and Bushel 1 to totals if no other quarters and bushels exist 

                if total_quarters == 0 and total_bushels == 0 and 'Quarters 1' in df.columns and 'Bushels 1' in df.columns: 

                    total_quarters = row['Quarters 1'] if not pd.isna(row['Quarters 1']) else 0.0 

                    total_bushels = row['Bushels 1'] if not pd.isna(row['Bushels 1']) else 0.0 

                 

                # Store the sums in the new columns 

                df.at[index, 'Total Quarters Except 1'] = total_quarters 

                df.at[index, 'Total Bushels Except 1'] = total_bushels 

 

               # Calculate Total Corn Cross-Check and Total Corn Manorial Account 

                df.at[index, 'Total Corn Cross-Check'] = total_quarters + (total_bushels / 8) 

                quarters_1 = row['Quarters 1'] if not pd.isna(row['Quarters 1']) else 0.0 

                bushels_1 = row['Bushels 1'] if not pd.isna(row['Bushels 1']) else 0.0 

                df.at[index, 'Total Corn Manorial Account'] = quarters_1 + (bushels_1 / 8) 

 

                # Calculate Corn Quantity Error 

                cross_check_total = df.at[index, 'Total Corn Cross-Check'] 

                manorial_account_total = df.at[index, 'Total Corn Manorial Account'] 

                df.at[index, 'Corn Quantity Error'] = cross_check_total - manorial_account_total 

 

 

            # Define the initial set of columns 

            ordered_columns = ['Heading', 'Sentence', 'Total in Pounds', 'Pounds', 'Shillings', 'Pence', 'Corn',  

                   'Total Quarters Except 1', 'Total Bushels Except 1', 'Total Corn Cross-Check',  

                   'Total Corn Manorial Account', 'Corn Quantity Error'] 

 

 

            # Determine the number of 'Quarters' and 'Bushels' columns 

            num_quarters_bushels_columns = max([int(col.split(' ')[-1]) for col in df.columns if 'Quarters' in col or 'Bushels' in col], default=0) 

 

            # Add 'Quarters' and 'Bushels' columns in pairs 

            for i in range(1, num_quarters_bushels_columns + 1): 

                ordered_columns.append(f'Quarters {i}') 

                ordered_columns.append(f'Bushels {i}') 

 

            # Determine the number of 'shilling a quarter' and 'pence a quarter' columns 

            num_quarter_columns = max([int(col.split(' ')[-1]) for col in df.columns if 'shilling a quarter' in col or 'pence a quarter' in col], default=0) 

 

            # Add 'shilling a quarter' and 'pence a quarter' columns in pairs 

            for i in range(1, num_quarter_columns + 1): 

                ordered_columns.append(f'shilling a quarter {i}') 

                ordered_columns.append(f'pence a quarter {i}') 

 

            # Determine the number of 'shilling a bushel' and 'pence a bushel' columns 

            num_bushel_columns = max([int(col.split(' ')[-1]) for col in df.columns if 'shilling a bushel' in col or 'pence a bushel' in col], default=0) 

 

            # Add 'shilling a bushel' and 'pence a bushel' columns in pairs 

            for i in range(1, num_bushel_columns + 1): 

                ordered_columns.append(f'shilling a bushel {i}') 

                ordered_columns.append(f'pence a bushel {i}') 

 

            # Reorder the DataFrame 

            df = df[ordered_columns] 

 

 

        elif sheet_name in ['Receipts', 'Expenses']: 

            # Process 'Receipts' and 'Expenses' sheets with your error calculation 

            df = insert_empty_rows_and_calculate_error(df) 

 

 

 

        # Save each processed sheet to the new Excel file 

        df.to_excel(writer, sheet_name=sheet_name, index=False) 

 

print("New Excel file created with modifications.") 

In [34]:
    ####################### SCRIPT 4  ####################### 
    
    # This script will sort Issues of the Grange Quantities
    # Goal 1: Sort out how much corn of each variety was produced vs. sold vs. sewn. etc.
    # Goal 2: Find out the value of all the corn produced! 
    
    
####################### BASIC SETUP #######################
import pandas as pd
import re

# List of document names
document_names = [
    "Adderbury", "Alresford", "AlresfordBorough", "Alverstoke", "Ashmansworth",
    "Beauworth", "Bentley", "BishopsFonthill", "BishopsSutton", "BishopsWaltham",
    "Bitterne", "Brightwell", "Burghclere", "Cams", "Cheriton", "Crawley", "Culham",
    "Downton", "DowntonBorough", "Droxford", "EastKnoyle", "EastMeon", "EastMeonChurch",
    "Ecchinswell", "Esher", "Farnham", "Gosport", "Hambledon", "Harwell", "Havant",
    "Highclere", "HindonBorough", "Holway", "Ivinghoe", "Kingston", "Marshalsea",
    "Merdon", "Morton", "Newtown", "NorthWaltham", "Otterford", "Overton",
    "OvertonBorough", "Poundisford", "Rimpton", "SokeofWinchester", "Southwark",
    "Staplegrove", "StGilesFair", "Taunton", "TauntonBorough", "TheChurch",
    "TrullBishopsHull", "Twyford", "Upton", "WalthamStLawrence", "Warfield", "Wargrave",
    "Warren", "WestWycombe", "Wield", "Witney", "WitneyBorough", "Wolvesey", "Woodhay"
]
    
# Load the Excel file
#file_path = "/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_sorted.xlsx"
#file_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script3/Ecchinswell3.xlsx'

# Directory path
base_input_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script3\\"
base_output_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script4\\"

for doc_name in document_names:
    input_file = f"{base_input_path}{doc_name}3.xlsx"
    output_file = f"{base_output_path}{doc_name}4.xlsx"

    ####################### IMPORT & DATAFRAME DEFINITION #######################
    # Load all sheets into a dictionary of DataFrames
    all_sheets_df = pd.read_excel(input_file, sheet_name=None)
        
    
    #I am basically isolating the Issues of the Grange sheet as its own dataframe; same for raw data
    issues_df = all_sheets_df['Issues of the Grange']
    raw_data_df = all_sheets_df['raw data']


    # Check if 'Issues of the Grange' is in the dictionary and if 'Sentence' is a column in the DataFrame
    if 'Issues of the Grange' in all_sheets_df and 'Sentence' in all_sheets_df['Issues of the Grange'].columns:
            issues_df = all_sheets_df['Issues of the Grange']
        
            ####################### COLUMNS ADJUSTED AND COPIED FROM RAW DATA #######################
            #All the action now happens on the issues_df which is the Issues of the Grange dataframe
            # Delete unwanted columns from 'Issues of the Grange'
            columns_to_delete = ["Total in Pounds", "Pounds", "Shillings", "Pence"]
            issues_df.drop(columns=columns_to_delete, inplace=True, errors='ignore')
            
            # Dynamically construct the columns to be copied from 'raw data'
            columns_to_copy = ["Total Quarters Except 1", "Total Bushels Except 1", 
                                "Total Corn Cross-Check", "Total Corn Manorial Account", 
                                "Corn Quantity Error"]

            ####################### EXTRACTING RELEVANT QUANTITIES BASED ON KEYWORDS #######################
            # Function to extract quarters and bushels based on specific keywords
            def extract_specific_quarters_bushels(sentence, keywords):
                # Convert sentence to lowercase for case-insensitive matching
                sentence_lower = sentence.lower()
            
                # Check if any keyword is in the sentence
                if any(keyword in sentence_lower for keyword in keywords):
                    # Regular expression to find patterns like 'xx quarters' or 'xx quarters yy bushels'
                    matches = re.findall(r'(\d+(?:\.\d+)?)\s+quarters(?:\s+(\d+(?:\.\d+)?)\s+bushels)?', sentence, re.IGNORECASE)
                    if matches:
                        # Summing up all found quantities
                        total_quarters = sum(float(quarter) for quarter, bushel in matches if quarter)
                        total_bushels = sum(float(bushel) if bushel else 0 for quarter, bushel in matches)
                        return total_quarters, total_bushels
                return 0, 0
        
            # Categories and their respective keywords
            categories = {
                'Sown': ['sown'],
                'Bought in': ['bought'],
                'Tithe': ['tithe'],
                'Manorial servants & livestock': ['delivered to the door-keeper', 'provision of the steward', 
                                                    'provision of the constable', 'for the liveries of', 'fodder of'],
                'Sold': ['sold'],
                'Account': ['same render account', 'from the whole issue', 'same renders account']  # Add this line
            }
        
            # Initialize columns for each category to ensure they exist even if no keywords are found
            for category in categories:
                issues_df[f'{category} Quarters'] = 0
                issues_df[f'{category} Bushels'] = 0
                issues_df[f'{category} Total Quarters'] = 0
        
        
        
            # Process each category (sown, tithe, bought, etc.) with correct quantities
            for category, keywords in categories.items():
                # Process each sentence to extract quarters and bushels, ensuring we handle the case where no data is found
                category_data = issues_df['Sentence'].apply(lambda x: extract_specific_quarters_bushels(x, keywords))
                issues_df[f'{category} Quarters'], issues_df[f'{category} Bushels'] = zip(*category_data)
                issues_df[f'{category} Total Quarters'] = issues_df[f'{category} Quarters'] + issues_df[f'{category} Bushels'] / 8
            
                # Fill NaNs right here to handle cases where no data might have led to NaNs
                issues_df[f'{category} Quarters'].fillna(0, inplace=True)
                issues_df[f'{category} Bushels'].fillna(0, inplace=True)
                issues_df[f'{category} Total Quarters'].fillna(0, inplace=True)
            
            # Print output to check columns created in issues_df
            print("Check1: Columns in issues_df after initialization:", issues_df.columns)
        
            # Extra processing step for account category
            # Check for both "bought" and "same render account" in the sentence for the "Account" category
            if category == 'Account':
                issues_df['bought only'] = issues_df['Sentence'].apply(lambda sentence: 1 if ('bought' in sentence.lower() and "same render account" in sentence.lower()) else 0)
            
            # Loop through each category and apply the extraction logic
            for category, keywords in categories.items():
                # Extracting data for the current category
                category_data = issues_df['Sentence'].apply(lambda x: extract_specific_quarters_bushels(x, keywords))
            
                # Unpack the data and add to DataFrame
                issues_df[f'{category} Quarters'], issues_df[f'{category} Bushels'] = zip(*category_data)
                # Convert all bushels to quarters for total
                issues_df[f'{category} Total Quarters'] = issues_df[f'{category} Quarters'] + issues_df[f'{category} Bushels'] / 8
            
            
            # List of corn types
            corn_types = ['wheat', 'mancorn', 'maslin', 'barley', 'first-grade malt', 'second-grade malt', 'peas', 'oats', 'oatmeal', 'vetches', 'rye', 'dredge', 'malt', 'meal', 'curall']
            
            
            # Append "Quarters x" and "Bushels x" where x is from 1 to 30
            for i in range(1, 31):
                quarter_column = f"Quarters {i}"
                bushel_column = f"Bushels {i}"
                if quarter_column in raw_data_df.columns:
                    columns_to_copy.append(quarter_column)
                if bushel_column in raw_data_df.columns:
                    columns_to_copy.append(bushel_column)

            ####################### MERGE ISSUES_DF WITH RAW DATA #######################
            # Merge 'Issues of the Grange' with the relevant columns from 'raw data' based on "Heading" and "Sentence"
            #these are: columns_to_copy = ["Total Quarters Except 1", "Total Bushels Except 1", "Total Corn Cross-Check", "Total Corn Manorial Account", "Corn Quantity Error"]
            all_columns = ["Heading", "Sentence"] + columns_to_copy
            issues_df = issues_df.merge(raw_data_df[all_columns], on=["Heading", "Sentence"], how="left")
            
            
            ####################### SAVE FINAL VERSION AND RENAME TO ISSUES GRANGE RAW #######################
            # Attach updated issues_df ('Issues of the Grange') back to the all_sheets_df
            # By using the .copy() method, you make sure that the df stored under "Issues of the Grange raw" is a separate copy from any further manipulations you might apply to issues_df after renaming it in the dictionary.
            all_sheets_df['Issues of the Grange'] = issues_df.copy()
            # Rename 'Issues of the Grange' to 'Issues of the Grange raw' in the all_sheets_df dictionary
            all_sheets_df['Issues of the Grange raw'] = all_sheets_df.pop('Issues of the Grange')
            print("Updated columns after processing: ", all_sheets_df['Issues of the Grange raw'].columns)
            
            
    
            ##################################################################################################
            #        GOAL 1: Sort out how much corn of each variety was produced vs. sold vs. sewn. etc.
            ##################################################################################################
            
            ####################### AGGREGATE INFORMATION IN ISSUES #######################
            # Step 1: Aggregate 'bought only' information - in the old issues_df
            bought_only_aggregation = issues_df.groupby('Heading')['bought only'].max().reset_index()
            bought_only_aggregation.rename(columns={'bought only': 'Bought Only Aggregated'}, inplace=True)
            
            # Convert this into a dictionary for easier mapping:
            bought_only_dict = bought_only_aggregation.set_index('Heading')['Bought Only Aggregated'].to_dict()
        
            ####################### NEW DICTIONARY CREATION #######################
            #Step 2: New df creation: Initialize the new DataFrame with appropriate data from issues_df
            ### Okay so here it gets interesting with the dataframes
            #I am creating a completely new and empty dataframe called new_sheets_df 
            #which I then attach to the full excel sheet dataframe with all the sheets
            #to do this I start with a list of dictionaries 
            ####################### ENSURE ALL COLUMNS EXIST #######################
            # Ensure that all potential columns are created in issues_df, filled with zeros if no data was found
            for category in categories:
                quarter_col = f'{category} Quarters'
                bushel_col = f'{category} Bushels'
                total_quarter_col = f'{category} Total Quarters'
                if quarter_col not in issues_df.columns:
                    issues_df[quarter_col] = 0
                if bushel_col not in issues_df.columns:
                    issues_df[bushel_col] = 0
                if total_quarter_col not in issues_df.columns:
                    issues_df[total_quarter_col] = 0
    
    
            ####################### CREATE NEW DATAFRAME FROM AGGREGATED DATA #######################
            # Initialize new_rows list for creating new_sheet_df
            new_rows = []
            
            # Populate new_rows with data aggregated based on headings and specific conditions
            for idx, row in issues_df.iterrows():
                new_row = {
                    'Gross Output': row['Heading'],
                    'Account Total Quarters': row.get('Account Total Quarters', 0),
                    'Bought Only Aggregated': bought_only_dict.get(row['Heading'], 0),
                    'Total Bought Quarters': row.get('Bought in Total Quarters', 0),
                    'Total Sold Quarters': row.get('Sold Total Quarters', 0)
                }
                new_rows.append(new_row)
    

            ####################### POPULATE DICTIONARIES & CREATE NEW DATAFRAME #######################
            # Populate the new dictionaries with aggregated rows for each corn type
            # Add rows for each corn type
            for corn_type in corn_types:
                bought_only_value = bought_only_aggregation.loc[bought_only_aggregation['Heading'].str.lower() == corn_type.lower(), 'Bought Only Aggregated'].max()
                new_row = {
                    'Gross Output': corn_type.capitalize(),
                    'Bought Only Aggregated': bought_only_value if pd.notnull(bought_only_value) else 0,
                    'Account Total Quarters': 0,
                    'Total Bought Quarters': 0,
                    'Total Sold Quarters': 0
                }
                new_rows.append(new_row)
        
            #creates a new DataFrame called new_sheet_df from a list of dictionaries stored in new_rows
            new_sheet_df = pd.DataFrame(new_rows)
            
            # By assigning new_sheet_df to all_sheets_df['Issues of the Grange'], 
            #you update the dictionary so that the key 'Issues of the Grange' now points to the new DataFrame. 
            all_sheets_df['Issues of the Grange'] = new_sheet_df
            
            
            # Specified columns to extract non-zero values from
            specified_columns = [
                "Account Quarters", "Account Bushels", "Account Total Quarters",
                "Bought in Quarters", "Bought in Bushels", "Bought in Total Quarters",
                "Sold Quarters",	"Sold Bushels",	"Sold Total Quarters"
            ]
            
            ####################### UPDATE COLUMNS #######################
            #This line would effectively update the 'Total Sold Quarters' column (or whichever is specified by column) 
            #in rows where the 'Gross Output' matches the specified corn_type with the non_zero_value.
            all_sheets_df['Issues of the Grange'].loc[all_sheets_df['Issues of the Grange']['Gross Output'].str.lower() == corn_type, column] = non_zero_value
            
        
        
    
            ####################### FILL IN NEW DATAFRAME WITH AGGREGATE INFORMATION FROM ISSUES #######################
            #Step 3: Only copying over aggregated data from issues_df to new_sheet
            #The purpose is to populate new_sheet_df with the maximum non-zero values from issues_df 
            #for each specified corn type and column. 
            #This is typically used to summarize data from a detailed DataFrame into a more aggregated form 
            for corn_type in corn_types:
                corn_row = issues_df[issues_df['Heading'].str.lower() == corn_type.lower()] #This line filters issues_df to find all rows where the 'Heading' matches the current corn_type in the loop, after converting both to lowercase for case-insensitive comparison. The result is stored in corn_row, which will be a DataFrame containing all rows that match the corn type.
                for column in specified_columns: #This nested loop iterates through a list of columns specified in specified_columns, before applying code below
                    non_zero_value = corn_row[corn_row[column] != 0][column].max()  #this finds the largest non-zero entry for the current corn type and column
                    if pd.notna(non_zero_value): #This checks if the non_zero_value is not NaN (i.e., it's a valid number). If true, the following line executes:
                        # Ensure we're updating the correct row in 'Issues of the Grange'
                        new_sheet_df.loc[new_sheet_df['Gross Output'].str.lower() == corn_type.lower(), column] = non_zero_value #This line updates new_sheet_df by setting the value in the specified column for rows where the 'Gross Output' matches the current corn type
            
            
            #Before performing operations on new_sheet_df, explicitly check for column existence:
            print("Columns available in new_sheet_df:", new_sheet_df.columns)
            if 'Bought in Total Quarters' not in new_sheet_df.columns:
                print("Error: 'Bought in Total Quarters' not found in new_sheet_df.")
                # Optionally add the column if it's critical for subsequent operations
                new_sheet_df['Bought in Total Quarters'] = 0
            else:
                new_sheet_df['Bought in Total Quarters'] = new_sheet_df['Bought in Total Quarters'].fillna(0)
            
        
            #Is this still needed?
            # Fill NaN values with 0 in the relevant columns before performing calculations
            new_sheet_df['Account Total Quarters'] = new_sheet_df['Account Total Quarters'].fillna(0)
            new_sheet_df['Bought in Total Quarters'] = new_sheet_df['Bought in Total Quarters'].fillna(0)
            new_sheet_df['Sold Total Quarters'] = new_sheet_df['Sold Total Quarters'].fillna(0)
        
        
            ####################### CALCULATE TOTAL CORN QUANTITY #######################
            #Step 4: 
            # Add 'Total Corn Quantity' column by summing 'Account Total Quarters' and 'Bought in Total Quarters'
            # Modify the 'Total Corn Quantity' calculation based on the condition
            def calculate_total_corn_quantity(row):
                if row['Bought Only Aggregated'] == 1:  # Assuming 'Bought Only Aggregated' is the correct column name after merging
                    # If 'bought only' is 1, do not include 'Bought in Total Quarters' in the sum
                    return row['Account Total Quarters']
                else:
                    # Else, sum 'Account Total Quarters' and 'Bought in Total Quarters'
                    return row['Account Total Quarters'] + row['Bought in Total Quarters']
            
            # Apply the function to each row
            new_sheet_df['Total Corn Quantity'] = new_sheet_df.apply(calculate_total_corn_quantity, axis=1)
            
            # Calculate 'Difference Produced+Bought-Sold' by subtracting 'Sold Total Quarters' from 'Total Corn Quantity'
            new_sheet_df['Difference Produced+Bought-Sold'] = new_sheet_df['Total Corn Quantity'] - new_sheet_df['Sold Total Quarters']
            
            
            # Make sure all expected columns exist, add them if they don't
            new_column_order = [
                'Gross Output', 'Bought Only Aggregated', 'Account Quarters', 'Account Bushels', 
                'Account Total Quarters', 'Bought in Quarters', 'Bought in Bushels', 
                'Bought in Total Quarters', 'Sold Quarters', 'Sold Bushels', 
                'Sold Total Quarters', 'Total Corn Quantity', 'Difference Produced+Bought-Sold'
            ]
        
            for column in new_column_order:
                if column not in new_sheet_df.columns:
                    new_sheet_df[column] = 0  # or appropriate default value
            
            # Reorder the DataFrame columns before saving
            new_sheet_df = new_sheet_df[new_column_order]
            
            # Make sure to replace `new_sheet_df` in the dictionary after updates
            all_sheets_df['Issues of the Grange'] = new_sheet_df
        
    
    
    
            ##################################################################################################
            #        GOAL 2: Extract sale of corn values and calculate average price per qrs
            ##################################################################################################
            
            #After extracting sale of corn values in a separate df, import into Issue of the Grange 
            #raw_data_df has been defined above, it's just the raw data sheet
            sale_of_corn_prices_df = raw_data_df[raw_data_df['Heading'].str.contains(
                "Sale of Corn|wheat, rye, dredge, peas and oats sold", case=False, na=False, regex=True)]
            
            #check to verify
            print("Rows in sale_of_corn_prices_df:", sale_of_corn_prices_df.shape[0])
            
            
            # Initial columns to include that are not dynamically named
            columns_to_include = [
                'Heading', 'Sentence', 'Total in Pounds', 'Pounds', 'Shillings', 'Pence', 'Corn',
                'Total Quarters Except 1', 'Total Bushels Except 1',
                'Total Corn Cross-Check', 'Total Corn Manorial Account', 'Corn Quantity Error'
            ]
        
            ####################### DYNAMICALLY ADD NEEDED COLUMNS #######################
            #Step1: Dynamically add 'Quarters x', 'Bushels x', 'shilling a quarter x', 'pence a quarter x'
            # Assuming the renaming has been correctly applied to sale_of_corn_prices_df
            for i in range(1, 31):  # Adjust the range as necessary, here it goes from 1 to 30
                columns_to_include.extend([
                    f'Quarters {i}', f'Bushels {i}',
                    f'shilling a quarter {i}', f'pence a quarter {i}'
                ])
            
            # Filter to include only columns that actually exist in raw_data_df
            columns_to_include = [col for col in columns_to_include if col in raw_data_df.columns]
            #Print to verify data is there
            print("Data in sale_of_corn_prices_df after filtering:")
            print(sale_of_corn_prices_df.head())
        
        
            # Now, use columns_to_include to filter the DataFrame
            sale_of_corn_prices_df = raw_data_df.loc[raw_data_df['Heading'].str.contains("Sale of Corn|wheat, rye, dredge, peas and oats sold", case=False, na=False), columns_to_include]
            
            sale_of_corn_prices_df = sale_of_corn_prices_df[columns_to_include].copy()
            
            #Add the New DataFrame to the Dictionary of DataFrames
            all_sheets_df['Sale of Corn Prices'] = sale_of_corn_prices_df
            
            rename_dict = {}
            # Renaming "Quarters 1" and "Bushels 1" to represent totals as "Quarters 0" and "Bushels 0"
            rename_dict['Quarters 1'] = 'Quarters 0'
            rename_dict['Bushels 1'] = 'Bushels 0'
            
            # Apply the initial renaming to sale_of_corn_prices_df
            sale_of_corn_prices_df.rename(columns=rename_dict, inplace=True)
        
    
            #renaming rest of bushels and quarters
            rename_dict_rest = {}
            for i in range(2, 31):  
                rename_dict_rest[f'Quarters {i}'] = f'Quarters {i-1}'
                rename_dict_rest[f'Bushels {i}'] = f'Bushels {i-1}'
            
            # Apply the renaming for the rest to sale_of_corn_prices_df
            sale_of_corn_prices_df.rename(columns=rename_dict_rest, inplace=True)
            
            # Now, verify the renaming by printing the columns
            #print(sale_of_corn_prices_df.columns)
        
        
            # Assuming the renaming has been done as previously described
            for i in range(0, 29):  # Adjust the range according to the actual number of quarters you have
                # Construct the column names
                shilling_col = f'shilling a quarter {i}'
                pence_col = f'pence a quarter {i}'
                pound_col = f'£ a quarter {i}'
    
                # Check if the shilling and pence columns exist before attempting the calculation
                if shilling_col in sale_of_corn_prices_df.columns and pence_col in sale_of_corn_prices_df.columns:
                    # Calculate "£ a quarter"
                    sale_of_corn_prices_df[pound_col] = (
                        sale_of_corn_prices_df[shilling_col] / 20 +
                        sale_of_corn_prices_df[pence_col] / 240
                    )
            
            # Calculate the total quarters sold by adding 'Quarters 0' to 'Bushels 0' divided by 8
            sale_of_corn_prices_df['Quarters Sold'] = sale_of_corn_prices_df['Quarters 0'] + sale_of_corn_prices_df['Bushels 0'] / 8
            
            # Check if "£ a quarter 1" is empty (NaN) and calculate the price per quarter only for those rows
            condition = pd.isna(sale_of_corn_prices_df['£ a quarter 1'])
            
            # Apply the calculation where the condition is True
            sale_of_corn_prices_df.loc[condition, '£ a quarter 1'] = sale_of_corn_prices_df['Total in Pounds'] / sale_of_corn_prices_df['Quarters Sold']
            
            # Verify the conditional update
            #print(sale_of_corn_prices_df[['Total in Pounds', 'Quarters 0', 'Bushels 0', 'Quarters Sold', '£ a quarter 1']])
        
    
    
            ####################### CALCULATE PRICES #######################
            #Calculating weight and non-weighted average prices per quarter
            # Initialize columns for weighted sums and total quarters
            weighted_sum = 0
            total_quarters = 0
            prices = []
            
            # Loop through each set of '£ a quarter x', 'Quarters x', and 'Bushels x'
            for i in range(1, 31):  # Adjust the range as necessary, here it goes from 1 to 30
                price_col = f'£ a quarter {i}'
                quarters_col = f'Quarters {i}'
                bushels_col = f'Bushels {i}'
                
                # Check if the columns exist in the DataFrame
                if price_col in df.columns and quarters_col in df.columns and bushels_col in df.columns:
                    # Convert bushels to quarters and calculate total quarters for this price
                    quarters = df[quarters_col] + df[bushels_col] / 8
                    # Calculate weighted sum for this price
                    weighted_sum += df[price_col] * quarters
                    # Update total quarters sold
                    total_quarters += quarters
                    # Collect prices for non-weighted average calculation
                    prices.append(df[price_col])

            # Calculate weighted average price per quarter
            def calculate_weighted_average(row):
                weighted_sum = 0
                total_quarters = 0
                prices_count = 0  # Track the number of non-empty price columns
            
                for i in range(1, 31):  # Adjust range as needed
                    price_col = f'£ a quarter {i}'
                    quarters_col = f'Quarters {i}'
                    bushels_col = f'Bushels {i}'
                    if price_col in row.index and quarters_col in row.index and bushels_col in row.index and not pd.isna(row[price_col]):
                        quarters = row[quarters_col] + row[bushels_col] / 8
                        weighted_sum += row[price_col] * quarters
                        total_quarters += quarters
                        prices_count += 1
            
                # If only one price is available, return it directly
                if prices_count == 1:
                    return row['£ a quarter 1']
        
                # Otherwise, return the calculated weighted average
                return weighted_sum / total_quarters if total_quarters else None
        
            def calculate_non_weighted_average(row):
                prices = [row[f'£ a quarter {i}'] for i in range(1, 31) if f'£ a quarter {i}' in row and not pd.isna(row[f'£ a quarter {i}'])]
                return sum(prices) / len(prices) if prices else None
            
            # Apply the functions row-wise
            sale_of_corn_prices_df['Weighted Average Price per Quarter'] = sale_of_corn_prices_df.apply(calculate_weighted_average, axis=1)
            sale_of_corn_prices_df['Non-Weighted Average Price per Quarter'] = sale_of_corn_prices_df.apply(calculate_non_weighted_average, axis=1)
            
        
            ####################### MERGE SALE OF CORN PRICES INTO NEW SHEET BASED ON CORN TYPE #######################
            #Merge prices based on 'Corn' and 'Gross Output' 
            # Convert 'Gross Output' in new_sheet_df and 'Corn' in sale_of_corn_prices_df to lowercase for a case-insensitive merge
            new_sheet_df['Gross Output lowercase'] = new_sheet_df['Gross Output'].str.lower()
            sale_of_corn_prices_df['Corn lowercase'] = sale_of_corn_prices_df['Corn'].str.lower()
            
            # Remove duplicates based on 'Corn lowercase', keeping only the first occurrence
            #This gets rid of aftersales (supercompotum) - but check with original to make sure. 
            sale_of_corn_prices_df = sale_of_corn_prices_df.drop_duplicates(subset=['Corn lowercase'], keep='first')
            
            
            # Perform the merge using the lowercase columns
            merged_df = pd.merge(new_sheet_df, sale_of_corn_prices_df[['Corn lowercase', 'Weighted Average Price per Quarter', 'Non-Weighted Average Price per Quarter']],
                                    left_on='Gross Output lowercase', right_on='Corn lowercase', how='left')
        
            # Optionally, you can drop the temporary lowercase columns after the merge if they are no longer needed
            merged_df.drop(columns=['Gross Output lowercase', 'Corn lowercase'], inplace=True)
            
            # Now, if you're observing duplicates in 'merged_df', it suggests there might be duplicates in 'new_sheet_df'
            # before merging or issues in how the merge keys uniquely identify rows.
            # Let's ensure there are no duplicates in 'new_sheet_df' before merging.
            new_sheet_df = new_sheet_df.drop_duplicates(subset=['Gross Output lowercase'], keep='first')
            
            # Perform the merge again with updated 'new_sheet_df'
            merged_df = pd.merge(new_sheet_df, sale_of_corn_prices_df[['Corn lowercase', 'Weighted Average Price per Quarter', 'Non-Weighted Average Price per Quarter']],
                                    left_on='Gross Output lowercase', right_on='Corn lowercase', how='left')
            
            # Drop the lowercase columns post-merge
            merged_df.drop(columns=['Gross Output lowercase', 'Corn lowercase'], inplace=True)
            
            # Update the main dictionary with the cleaned, merged data
            all_sheets_df['Issues of the Grange'] = merged_df
        
        
            ####################### CALCULATE TOTAL VALUE OF PRODUCTION #######################
            # Calculate 'Value of Total Production weighted'
            merged_df['Value of Total Production weighted'] = merged_df['Account Total Quarters'] * merged_df['Weighted Average Price per Quarter']
            
            # Calculate 'Value of Total Production unweighted'
            merged_df['Value of Total Production unweighted'] = merged_df['Account Total Quarters'] * merged_df['Non-Weighted Average Price per Quarter']
            
        
            #This makes sure that the merged variables actually show up
            new_sheet_df = merged_df
            
            # Make sure to update 'Issues of the Grange' in all_sheets_df with the modified new_sheet_df
            all_sheets_df['Issues of the Grange'] = new_sheet_df
    


    else:
        # If 'Issues of the Grange' is not present or 'Sentence' column is missing,
        # notify and skip processing related to it
        print("'Sentence' column not found in 'Issues of the Grange'. Skipping related processing.")
        

        ####################### FINAL SAVE #######################
        # Save all DataFrames to a new Excel file
        #output_file = "/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_sorted2.xlsx"
        #output_file = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script4/Ecchinswell4.xlsx'
            
                
        with pd.ExcelWriter(output_file) as writer:
            for sheet_name, df in all_sheets_df.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                        
        # Optionally add a print statement to confirm which file has been saved
        print(f"Saved processed data to {output_file}")

C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  issues_df[f'{category} Quarters'].fillna(0, inplace=True)
C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:109: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Check1: Columns in issues_df after initialization: Index(['Heading', 'Sentence', 'Quarters 1', 'Bushels 1', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servants & livestock Bushels',
       'Manorial servants & livestock Total Quarters', 'Sold Quarters',
       'Sold Bushels', 'Sold Total Quarters', 'Account Quarters',
       'Account Bushels', 'Account Total Quarters'],
      dtype='object')
Updated columns after processing:  Index(['Heading', 'Sentence', 'Quarters 1_x', 'Bushels 1_x', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servan

C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  issues_df[f'{category} Quarters'].fillna(0, inplace=True)
C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:109: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Check1: Columns in issues_df after initialization: Index(['Heading', 'Sentence', 'Quarters 1', 'Bushels 1', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servants & livestock Bushels',
       'Manorial servants & livestock Total Quarters', 'Sold Quarters',
       'Sold Bushels', 'Sold Total Quarters', 'Account Quarters',
       'Account Bushels', 'Account Total Quarters'],
      dtype='object')
Updated columns after processing:  Index(['Heading', 'Sentence', 'Quarters 1_x', 'Bushels 1_x', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servan

C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  issues_df[f'{category} Quarters'].fillna(0, inplace=True)
C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:109: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

...

Check1: Columns in issues_df after initialization: Index(['Heading', 'Sentence', 'Quarters 1', 'Bushels 1', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servants & livestock Bushels',
       'Manorial servants & livestock Total Quarters', 'Sold Quarters',
       'Sold Bushels', 'Sold Total Quarters', 'Account Quarters',
       'Account Bushels', 'Account Total Quarters'],
      dtype='object')
Updated columns after processing:  Index(['Heading', 'Sentence', 'Quarters 1_x', 'Bushels 1_x', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servan

C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  issues_df[f'{category} Quarters'].fillna(0, inplace=True)
C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:109: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Check1: Columns in issues_df after initialization: Index(['Heading', 'Sentence', 'Quarters 1', 'Bushels 1', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servants & livestock Bushels',
       'Manorial servants & livestock Total Quarters', 'Sold Quarters',
       'Sold Bushels', 'Sold Total Quarters', 'Account Quarters',
       'Account Bushels', 'Account Total Quarters'],
      dtype='object')
Updated columns after processing:  Index(['Heading', 'Sentence', 'Quarters 1_x', 'Bushels 1_x', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servan

C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  issues_df[f'{category} Quarters'].fillna(0, inplace=True)
C:\Users\kubak\AppData\Local\Temp\ipykernel_40880\3779115222.py:109: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

KeyError: 'Sold Total Quarters'

In [33]:
    ####################### SCRIPT 4  ####################### 
    
    # This script will sort Issues of the Grange Quantities
    # Goal 1: Sort out how much corn of each variety was produced vs. sold vs. sewn. etc.
    # Goal 2: Find out the value of all the corn produced! 
    
    
####################### BASIC SETUP #######################
import pandas as pd
import re

# List of document names
document_names = [
    "Adderbury", "Alresford", "AlresfordBorough", "Alverstoke", "Ashmansworth",
    "Beauworth", "Bentley", "BishopsFonthill", "BishopsSutton", "BishopsWaltham",
    "Bitterne", "Brightwell", "Burghclere", "Cams", "Cheriton", "Crawley", "Culham",
    "Downton", "DowntonBorough", "Droxford", "EastKnoyle", "EastMeon", "EastMeonChurch",
    "Ecchinswell", "Esher", "Farnham", "Gosport", "Hambledon", "Harwell", "Havant",
    "Highclere", "HindonBorough", "Holway", "Ivinghoe", "Kingston", "Marshalsea",
    "Merdon", "Morton", "Newtown", "NorthWaltham", "Otterford", "Overton",
    "OvertonBorough", "Poundisford", "Rimpton", "SokeofWinchester", "Southwark",
    "Staplegrove", "StGilesFair", "Taunton", "TauntonBorough", "TheChurch",
    "TrullBishopsHull", "Twyford", "Upton", "WalthamStLawrence", "Warfield", "Wargrave",
    "Warren", "WestWycombe", "Wield", "Witney", "WitneyBorough", "Wolvesey", "Woodhay"
]
    
# Load the Excel file
#file_path = "/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_sorted.xlsx"
#file_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script3/Ecchinswell3.xlsx'

# Directory path
base_input_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script3\\"
base_output_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script4\\"

for doc_name in document_names:
        input_file = f"{base_input_path}{doc_name}3.xlsx"
        output_file = f"{base_output_path}{doc_name}4.xlsx"
    
        ####################### IMPORT & DATAFRAME DEFINITION #######################
        # Load all sheets into a dictionary of DataFrames
        all_sheets_df = pd.read_excel(input_file, sheet_name=None)
            
        
        #I am basically isolating the Issues of the Grange sheet as its own dataframe; same for raw data
        issues_df = all_sheets_df['Issues of the Grange'].copy()
        raw_data_df = all_sheets_df['raw data'].copy()

    
    
        # Check if 'Issues of the Grange' is in the dictionary and if 'Sentence' is a column in the DataFrame
        if 'Issues of the Grange' in all_sheets_df and 'Sentence' in all_sheets_df['Issues of the Grange'].columns:
                issues_df = all_sheets_df['Issues of the Grange']
            
                ####################### COLUMNS ADJUSTED AND COPIED FROM RAW DATA #######################
                #All the action now happens on the issues_df which is the Issues of the Grange dataframe
                # Delete unwanted columns from 'Issues of the Grange'
                columns_to_delete = ["Total in Pounds", "Pounds", "Shillings", "Pence"]
                issues_df.drop(columns=columns_to_delete, inplace=True, errors='ignore')
                
                # Dynamically construct the columns to be copied from 'raw data'
                columns_to_copy = ["Total Quarters Except 1", "Total Bushels Except 1", 
                                   "Total Corn Cross-Check", "Total Corn Manorial Account", 
                                   "Corn Quantity Error"]
    
                ####################### EXTRACTING RELEVANT QUANTITIES BASED ON KEYWORDS #######################
                # Function to extract quarters and bushels based on specific keywords
                def extract_specific_quarters_bushels(sentence, keywords):
                    # Convert sentence to lowercase for case-insensitive matching
                    sentence_lower = sentence.lower()
                
                    # Check if any keyword is in the sentence
                    if any(keyword in sentence_lower for keyword in keywords):
                        # Regular expression to find patterns like 'xx quarters' or 'xx quarters yy bushels'
                        matches = re.findall(r'(\d+(?:\.\d+)?)\s+quarters(?:\s+(\d+(?:\.\d+)?)\s+bushels)?', sentence, re.IGNORECASE)
                        if matches:
                            # Summing up all found quantities
                            total_quarters = sum(float(quarter) for quarter, bushel in matches if quarter)
                            total_bushels = sum(float(bushel) if bushel else 0 for quarter, bushel in matches)
                            return total_quarters, total_bushels
                    return 0, 0
            
                # Categories and their respective keywords
                categories = {
                    'Sown': ['sown'],
                    'Bought in': ['bought'],
                    'Tithe': ['tithe'],
                    'Manorial servants & livestock': ['delivered to the door-keeper', 'provision of the steward', 
                                                      'provision of the constable', 'for the liveries of', 'fodder of'],
                    'Sold': ['sold'],
                    'Account': ['same render account', 'from the whole issue', 'same renders account']  # Add this line
                }
            
                # Initialize columns for each category to ensure they exist even if no keywords are found
                for category in categories:
                    issues_df[f'{category} Quarters'] = 0
                    issues_df[f'{category} Bushels'] = 0
                    issues_df[f'{category} Total Quarters'] = 0
            
            
            
                # Process each category (sown, tithe, bought, etc.) with correct quantities
                for category, keywords in categories.items():
                    # Process each sentence to extract quarters and bushels, ensuring we handle the case where no data is found
                    category_data = issues_df['Sentence'].apply(lambda x: extract_specific_quarters_bushels(x, keywords))
                    issues_df[f'{category} Quarters'], issues_df[f'{category} Bushels'] = zip(*category_data)
                    issues_df[f'{category} Total Quarters'] = issues_df[f'{category} Quarters'] + issues_df[f'{category} Bushels'] / 8
                
                    # Fill NaNs right here to handle cases where no data might have led to NaNs
                    issues_df[f'{category} Quarters'] = issues_df[f'{category} Quarters'].fillna(0)
                    issues_df[f'{category} Bushels']  = issues_df[f'{category} Bushels'].fillna(0)
                    issues_df[f'{category} Total Quarters'] = issues_df[f'{category} Total Quarters'].fillna(0)

                
                # Print output to check columns created in issues_df
                print("Check1: Columns in issues_df after initialization:", issues_df.columns)
            
                # Extra processing step for account category
                # Check for both "bought" and "same render account" in the sentence for the "Account" category
                if category == 'Account':
                    issues_df['bought only'] = (
                        issues_df['Sentence'].str.contains('bought', case=False, na=False) &
                        issues_df['Sentence'].str.contains('same render account|same renders account', case=False, na=False)
                    ).astype(int)

                # Loop through each category and apply the extraction logic
                for category, keywords in categories.items():
                    # Extracting data for the current category
                    category_data = issues_df['Sentence'].apply(lambda x: extract_specific_quarters_bushels(x, keywords))
                
                    # Unpack the data and add to DataFrame
                    issues_df[f'{category} Quarters'], issues_df[f'{category} Bushels'] = zip(*category_data)
                    # Convert all bushels to quarters for total
                    issues_df[f'{category} Total Quarters'] = issues_df[f'{category} Quarters'] + issues_df[f'{category} Bushels'] / 8
                
                
                # List of corn types
                corn_types = ['wheat', 'mancorn', 'maslin', 'barley', 'first-grade malt', 'second-grade malt', 'peas', 'oats', 'oatmeal', 'vetches', 'rye', 'dredge', 'malt', 'meal', 'curall']
                
                
                # Append "Quarters x" and "Bushels x" where x is from 1 to 30
                for i in range(1, 31):
                    quarter_column = f"Quarters {i}"
                    bushel_column = f"Bushels {i}"
                    if quarter_column in raw_data_df.columns:
                        columns_to_copy.append(quarter_column)
                    if bushel_column in raw_data_df.columns:
                        columns_to_copy.append(bushel_column)
    
                ####################### MERGE ISSUES_DF WITH RAW DATA #######################
                # Merge 'Issues of the Grange' with the relevant columns from 'raw data' based on "Heading" and "Sentence"
                #these are: columns_to_copy = ["Total Quarters Except 1", "Total Bushels Except 1", "Total Corn Cross-Check", "Total Corn Manorial Account", "Corn Quantity Error"]
                all_columns = ["Heading", "Sentence"] + columns_to_copy
                issues_df = issues_df.merge(raw_data_df[all_columns], on=["Heading", "Sentence"], how="left")
                
                
                ####################### SAVE FINAL VERSION AND RENAME TO ISSUES GRANGE RAW #######################
                # Attach updated issues_df ('Issues of the Grange') back to the all_sheets_df
                # By using the .copy() method, you make sure that the df stored under "Issues of the Grange raw" is a separate copy from any further manipulations you might apply to issues_df after renaming it in the dictionary.
                all_sheets_df['Issues of the Grange'] = issues_df.copy()
                # Rename 'Issues of the Grange' to 'Issues of the Grange raw' in the all_sheets_df dictionary
                all_sheets_df['Issues of the Grange raw'] = all_sheets_df.pop('Issues of the Grange')
                print("Updated columns after processing: ", all_sheets_df['Issues of the Grange raw'].columns)
                
                
        
                ##################################################################################################
                #        GOAL 1: Sort out how much corn of each variety was produced vs. sold vs. sewn. etc.
                ##################################################################################################
                
                ####################### AGGREGATE INFORMATION IN ISSUES #######################
                # Step 1: Aggregate 'bought only' information - in the old issues_df
                bought_only_aggregation = issues_df.groupby('Heading')['bought only'].max().reset_index()
                bought_only_aggregation.rename(columns={'bought only': 'Bought Only Aggregated'}, inplace=True)
                
                # Convert this into a dictionary for easier mapping:
                bought_only_dict = bought_only_aggregation.set_index('Heading')['Bought Only Aggregated'].to_dict()
            
                ####################### NEW DICTIONARY CREATION #######################
                #Step 2: New df creation: Initialize the new DataFrame with appropriate data from issues_df
                ### Okay so here it gets interesting with the dataframes
                #I am creating a completely new and empty dataframe called new_sheets_df 
                #which I then attach to the full excel sheet dataframe with all the sheets
                #to do this I start with a list of dictionaries 
                ####################### ENSURE ALL COLUMNS EXIST #######################
                # Ensure that all potential columns are created in issues_df, filled with zeros if no data was found
                for category in categories:
                    quarter_col = f'{category} Quarters'
                    bushel_col = f'{category} Bushels'
                    total_quarter_col = f'{category} Total Quarters'
                    if quarter_col not in issues_df.columns:
                        issues_df[quarter_col] = 0
                    if bushel_col not in issues_df.columns:
                        issues_df[bushel_col] = 0
                    if total_quarter_col not in issues_df.columns:
                        issues_df[total_quarter_col] = 0
        
        
                ####################### CREATE NEW DATAFRAME FROM AGGREGATED DATA #######################
                # Initialize new_rows list for creating new_sheet_df
                new_rows = []
                
                # Populate new_rows with data aggregated based on headings and specific conditions
                for idx, row in issues_df.iterrows():
                    new_row = {
                        'Gross Output': row['Heading'],
                        'Account Total Quarters': row.get('Account Total Quarters', 0),
                        'Bought Only Aggregated': bought_only_dict.get(row['Heading'], 0),
                        'Total Bought Quarters': row.get('Bought in Total Quarters', 0),
                        'Total Sold Quarters': row.get('Sold Total Quarters', 0)
                    }
                    new_rows.append(new_row)
        
    
                ####################### POPULATE DICTIONARIES & CREATE NEW DATAFRAME #######################
                # Populate the new dictionaries with aggregated rows for each corn type
                # Add rows for each corn type
                for corn_type in corn_types:
                    bought_only_value = bought_only_aggregation.loc[bought_only_aggregation['Heading'].str.lower() == corn_type.lower(), 'Bought Only Aggregated'].max()
                    new_row = {
                        'Gross Output': corn_type.capitalize(),
                        'Bought Only Aggregated': bought_only_value if pd.notnull(bought_only_value) else 0,
                        'Account Total Quarters': 0,
                        'Total Bought Quarters': 0,
                        'Total Sold Quarters': 0
                    }
                    new_rows.append(new_row)
            
                #creates a new DataFrame called new_sheet_df from a list of dictionaries stored in new_rows
                new_sheet_df = pd.DataFrame(new_rows)
                
                new_sheet_df.rename(
                    columns={
                        'Total Sold Quarters': 'Sold Total Quarters',
                        'Total Bought Quarters': 'Bought in Total Quarters'
                    },
                    inplace=True
                )

                # By assigning new_sheet_df to all_sheets_df['Issues of the Grange'], 
                #you update the dictionary so that the key 'Issues of the Grange' now points to the new DataFrame. 
                all_sheets_df['Issues of the Grange'] = new_sheet_df
                
                
                # Specified columns to extract non-zero values from
                specified_columns = [
                    "Account Quarters", "Account Bushels", "Account Total Quarters",
                    "Bought in Quarters", "Bought in Bushels", "Bought in Total Quarters",
                    "Sold Quarters",	"Sold Bushels",	"Sold Total Quarters"
                ]
                
                # ####################### UPDATE COLUMNS #######################
                # #This line would effectively update the 'Total Sold Quarters' column (or whichever is specified by column) 
                # #in rows where the 'Gross Output' matches the specified corn_type with the non_zero_value.
                # issues_out = all_sheets_df['Issues of the Grange'].copy()

                # mask = issues_out['Gross Output'].str.lower().eq(corn_type.lower())
                # issues_out.loc[mask, column] = non_zero_value

                # all_sheets_df['Issues of the Grange'] = issues_out

                
            
            
        
                ####################### FILL IN NEW DATAFRAME WITH AGGREGATE INFORMATION FROM ISSUES #######################
                #Step 3: Only copying over aggregated data from issues_df to new_sheet
                #The purpose is to populate new_sheet_df with the maximum non-zero values from issues_df 
                #for each specified corn type and column. 
                #This is typically used to summarize data from a detailed DataFrame into a more aggregated form 
                for corn_type in corn_types:
                    corn_row = issues_df[issues_df['Heading'].str.lower() == corn_type.lower()] #This line filters issues_df to find all rows where the 'Heading' matches the current corn_type in the loop, after converting both to lowercase for case-insensitive comparison. The result is stored in corn_row, which will be a DataFrame containing all rows that match the corn type.
                    for column in specified_columns: #This nested loop iterates through a list of columns specified in specified_columns, before applying code below
                        non_zero_value = corn_row[corn_row[column] != 0][column].max()  #this finds the largest non-zero entry for the current corn type and column
                        if pd.notna(non_zero_value): #This checks if the non_zero_value is not NaN (i.e., it's a valid number). If true, the following line executes:
                            # Ensure we're updating the correct row in 'Issues of the Grange'
                            new_sheet_df.loc[new_sheet_df['Gross Output'].str.lower() == corn_type.lower(), column] = non_zero_value #This line updates new_sheet_df by setting the value in the specified column for rows where the 'Gross Output' matches the current corn type
                
                
                #Before performing operations on new_sheet_df, explicitly check for column existence:
                print("Columns available in new_sheet_df:", new_sheet_df.columns)
                if 'Bought in Total Quarters' not in new_sheet_df.columns:
                    print("Error: 'Bought in Total Quarters' not found in new_sheet_df.")
                    # Optionally add the column if it's critical for subsequent operations
                    new_sheet_df['Bought in Total Quarters'] = 0
                else:
                    new_sheet_df['Bought in Total Quarters'] = new_sheet_df['Bought in Total Quarters'].fillna(0)
                
            
                #Is this still needed?
                # Fill NaN values with 0 in the relevant columns before performing calculations
                new_sheet_df['Account Total Quarters'] = new_sheet_df['Account Total Quarters'].fillna(0)
                new_sheet_df['Bought in Total Quarters'] = new_sheet_df['Bought in Total Quarters'].fillna(0)
                new_sheet_df['Sold Total Quarters'] = new_sheet_df['Sold Total Quarters'].fillna(0)
            
            
                ####################### CALCULATE TOTAL CORN QUANTITY #######################
                #Step 4: 
                # Add 'Total Corn Quantity' column by summing 'Account Total Quarters' and 'Bought in Total Quarters'
                # Modify the 'Total Corn Quantity' calculation based on the condition
                def calculate_total_corn_quantity(row):
                    if row['Bought Only Aggregated'] == 1:  # Assuming 'Bought Only Aggregated' is the correct column name after merging
                        # If 'bought only' is 1, do not include 'Bought in Total Quarters' in the sum
                        return row['Account Total Quarters']
                    else:
                        # Else, sum 'Account Total Quarters' and 'Bought in Total Quarters'
                        return row['Account Total Quarters'] + row['Bought in Total Quarters']
                
                # Apply the function to each row
                new_sheet_df['Total Corn Quantity'] = new_sheet_df.apply(calculate_total_corn_quantity, axis=1)
                
                # Calculate 'Difference Produced+Bought-Sold' by subtracting 'Sold Total Quarters' from 'Total Corn Quantity'
                new_sheet_df['Difference Produced+Bought-Sold'] = new_sheet_df['Total Corn Quantity'] - new_sheet_df['Sold Total Quarters']
                
                
                # Make sure all expected columns exist, add them if they don't
                new_column_order = [
                    'Gross Output', 'Bought Only Aggregated', 'Account Quarters', 'Account Bushels', 
                    'Account Total Quarters', 'Bought in Quarters', 'Bought in Bushels', 
                    'Bought in Total Quarters', 'Sold Quarters', 'Sold Bushels', 
                    'Sold Total Quarters', 'Total Corn Quantity', 'Difference Produced+Bought-Sold'
                ]
            
                for column in new_column_order:
                    if column not in new_sheet_df.columns:
                        new_sheet_df[column] = 0  # or appropriate default value
                
                # Reorder the DataFrame columns before saving
                new_sheet_df = new_sheet_df[new_column_order]
                
                # Make sure to replace `new_sheet_df` in the dictionary after updates
                all_sheets_df['Issues of the Grange'] = new_sheet_df
            
        
        
        
                ##################################################################################################
                #        GOAL 2: Extract sale of corn values and calculate average price per qrs
                ##################################################################################################
                
                #After extracting sale of corn values in a separate df, import into Issue of the Grange 
                #raw_data_df has been defined above, it's just the raw data sheet
                sale_of_corn_prices_df = raw_data_df[raw_data_df['Heading'].str.contains(
                    "Sale of Corn|wheat, rye, dredge, peas and oats sold", case=False, na=False, regex=True)]
                
                #check to verify
                print("Rows in sale_of_corn_prices_df:", sale_of_corn_prices_df.shape[0])
                
                
                # Initial columns to include that are not dynamically named
                columns_to_include = [
                    'Heading', 'Sentence', 'Total in Pounds', 'Pounds', 'Shillings', 'Pence', 'Corn',
                    'Total Quarters Except 1', 'Total Bushels Except 1',
                    'Total Corn Cross-Check', 'Total Corn Manorial Account', 'Corn Quantity Error'
                ]
            
                ####################### DYNAMICALLY ADD NEEDED COLUMNS #######################
                #Step1: Dynamically add 'Quarters x', 'Bushels x', 'shilling a quarter x', 'pence a quarter x'
                # Assuming the renaming has been correctly applied to sale_of_corn_prices_df
                for i in range(1, 31):  # Adjust the range as necessary, here it goes from 1 to 30
                    columns_to_include.extend([
                        f'Quarters {i}', f'Bushels {i}',
                        f'shilling a quarter {i}', f'pence a quarter {i}'
                    ])
                
                # Filter to include only columns that actually exist in raw_data_df
                columns_to_include = [col for col in columns_to_include if col in raw_data_df.columns]
                #Print to verify data is there
                print("Data in sale_of_corn_prices_df after filtering:")
                print(sale_of_corn_prices_df.head())
            
            
                # Now, use columns_to_include to filter the DataFrame
                sale_of_corn_prices_df = raw_data_df.loc[raw_data_df['Heading'].str.contains("Sale of Corn|wheat, rye, dredge, peas and oats sold", case=False, na=False), columns_to_include]
                
                sale_of_corn_prices_df = sale_of_corn_prices_df[columns_to_include].copy()
                
                #Add the New DataFrame to the Dictionary of DataFrames
                all_sheets_df['Sale of Corn Prices'] = sale_of_corn_prices_df
                
                rename_dict = {}
                # Renaming "Quarters 1" and "Bushels 1" to represent totals as "Quarters 0" and "Bushels 0"
                rename_dict['Quarters 1'] = 'Quarters 0'
                rename_dict['Bushels 1'] = 'Bushels 0'
                
                # Apply the initial renaming to sale_of_corn_prices_df
                sale_of_corn_prices_df.rename(columns=rename_dict, inplace=True)
            
        
                #renaming rest of bushels and quarters
                rename_dict_rest = {}
                for i in range(2, 31):  
                    rename_dict_rest[f'Quarters {i}'] = f'Quarters {i-1}'
                    rename_dict_rest[f'Bushels {i}'] = f'Bushels {i-1}'
                
                # Apply the renaming for the rest to sale_of_corn_prices_df
                sale_of_corn_prices_df.rename(columns=rename_dict_rest, inplace=True)
                
                # Now, verify the renaming by printing the columns
                #print(sale_of_corn_prices_df.columns)
            
            
                # Assuming the renaming has been done as previously described
                for i in range(0, 29):  # Adjust the range according to the actual number of quarters you have
                    # Construct the column names
                    shilling_col = f'shilling a quarter {i}'
                    pence_col = f'pence a quarter {i}'
                    pound_col = f'£ a quarter {i}'
        
                    # Check if the shilling and pence columns exist before attempting the calculation
                    if shilling_col in sale_of_corn_prices_df.columns and pence_col in sale_of_corn_prices_df.columns:
                        # Calculate "£ a quarter"
                        sale_of_corn_prices_df[pound_col] = (
                            sale_of_corn_prices_df[shilling_col] / 20 +
                            sale_of_corn_prices_df[pence_col] / 240
                        )
                
                # Calculate the total quarters sold by adding 'Quarters 0' to 'Bushels 0' divided by 8
                sale_of_corn_prices_df['Quarters Sold'] = sale_of_corn_prices_df['Quarters 0'] + sale_of_corn_prices_df['Bushels 0'] / 8
                
                # Check if "£ a quarter 1" is empty (NaN) and calculate the price per quarter only for those rows
                condition = pd.isna(sale_of_corn_prices_df['£ a quarter 1'])
                
                # Apply the calculation where the condition is True
                sale_of_corn_prices_df.loc[condition, '£ a quarter 1'] = sale_of_corn_prices_df['Total in Pounds'] / sale_of_corn_prices_df['Quarters Sold']
                
                # Verify the conditional update
                #print(sale_of_corn_prices_df[['Total in Pounds', 'Quarters 0', 'Bushels 0', 'Quarters Sold', '£ a quarter 1']])
            
        
        
                ####################### CALCULATE PRICES #######################
                #Calculating weight and non-weighted average prices per quarter
                # Initialize columns for weighted sums and total quarters
                weighted_sum = 0
                total_quarters = 0
                prices = []
                
                # Loop through each set of '£ a quarter x', 'Quarters x', and 'Bushels x'
                for i in range(1, 31):  # Adjust the range as necessary, here it goes from 1 to 30
                    price_col = f'£ a quarter {i}'
                    quarters_col = f'Quarters {i}'
                    bushels_col = f'Bushels {i}'
                    
                    # Check if the columns exist in the DataFrame
                    if price_col in df.columns and quarters_col in df.columns and bushels_col in df.columns:
                        # Convert bushels to quarters and calculate total quarters for this price
                        quarters = df[quarters_col] + df[bushels_col] / 8
                        # Calculate weighted sum for this price
                        weighted_sum += df[price_col] * quarters
                        # Update total quarters sold
                        total_quarters += quarters
                        # Collect prices for non-weighted average calculation
                        prices.append(df[price_col])
    
                # Calculate weighted average price per quarter
                def calculate_weighted_average(row):
                    weighted_sum = 0
                    total_quarters = 0
                    prices_count = 0  # Track the number of non-empty price columns
                
                    for i in range(1, 31):  # Adjust range as needed
                        price_col = f'£ a quarter {i}'
                        quarters_col = f'Quarters {i}'
                        bushels_col = f'Bushels {i}'
                        if price_col in row.index and quarters_col in row.index and bushels_col in row.index and not pd.isna(row[price_col]):
                            quarters = row[quarters_col] + row[bushels_col] / 8
                            weighted_sum += row[price_col] * quarters
                            total_quarters += quarters
                            prices_count += 1
                
                    # If only one price is available, return it directly
                    if prices_count == 1:
                        return row['£ a quarter 1']
            
                    # Otherwise, return the calculated weighted average
                    return weighted_sum / total_quarters if total_quarters else None
            
                def calculate_non_weighted_average(row):
                    prices = [row[f'£ a quarter {i}'] for i in range(1, 31) if f'£ a quarter {i}' in row and not pd.isna(row[f'£ a quarter {i}'])]
                    return sum(prices) / len(prices) if prices else None
                
                # Apply the functions row-wise
                sale_of_corn_prices_df['Weighted Average Price per Quarter'] = sale_of_corn_prices_df.apply(calculate_weighted_average, axis=1)
                sale_of_corn_prices_df['Non-Weighted Average Price per Quarter'] = sale_of_corn_prices_df.apply(calculate_non_weighted_average, axis=1)
                
            
                ####################### MERGE SALE OF CORN PRICES INTO NEW SHEET BASED ON CORN TYPE #######################
                #Merge prices based on 'Corn' and 'Gross Output' 
                # Convert 'Gross Output' in new_sheet_df and 'Corn' in sale_of_corn_prices_df to lowercase for a case-insensitive merge
                new_sheet_df['Gross Output lowercase'] = new_sheet_df['Gross Output'].str.lower()
                sale_of_corn_prices_df['Corn lowercase'] = sale_of_corn_prices_df['Corn'].str.lower()
                
                # Remove duplicates based on 'Corn lowercase', keeping only the first occurrence
                #This gets rid of aftersales (supercompotum) - but check with original to make sure. 
                sale_of_corn_prices_df = sale_of_corn_prices_df.drop_duplicates(subset=['Corn lowercase'], keep='first')
                
                
                # Perform the merge using the lowercase columns
                merged_df = pd.merge(new_sheet_df, sale_of_corn_prices_df[['Corn lowercase', 'Weighted Average Price per Quarter', 'Non-Weighted Average Price per Quarter']],
                                     left_on='Gross Output lowercase', right_on='Corn lowercase', how='left')
            
                # Optionally, you can drop the temporary lowercase columns after the merge if they are no longer needed
                merged_df.drop(columns=['Gross Output lowercase', 'Corn lowercase'], inplace=True)
                
                # Now, if you're observing duplicates in 'merged_df', it suggests there might be duplicates in 'new_sheet_df'
                # before merging or issues in how the merge keys uniquely identify rows.
                # Let's ensure there are no duplicates in 'new_sheet_df' before merging.
                new_sheet_df = new_sheet_df.drop_duplicates(subset=['Gross Output lowercase'], keep='first')
                
                # Perform the merge again with updated 'new_sheet_df'
                merged_df = pd.merge(new_sheet_df, sale_of_corn_prices_df[['Corn lowercase', 'Weighted Average Price per Quarter', 'Non-Weighted Average Price per Quarter']],
                                     left_on='Gross Output lowercase', right_on='Corn lowercase', how='left')
                
                # Drop the lowercase columns post-merge
                merged_df.drop(columns=['Gross Output lowercase', 'Corn lowercase'], inplace=True)
                
                # Update the main dictionary with the cleaned, merged data
                all_sheets_df['Issues of the Grange'] = merged_df
            
            
                ####################### CALCULATE TOTAL VALUE OF PRODUCTION #######################
                # Calculate 'Value of Total Production weighted'
                merged_df['Value of Total Production weighted'] = merged_df['Account Total Quarters'] * merged_df['Weighted Average Price per Quarter']
                
                # Calculate 'Value of Total Production unweighted'
                merged_df['Value of Total Production unweighted'] = merged_df['Account Total Quarters'] * merged_df['Non-Weighted Average Price per Quarter']
                
            
                #This makes sure that the merged variables actually show up
                new_sheet_df = merged_df
                
                # Make sure to update 'Issues of the Grange' in all_sheets_df with the modified new_sheet_df
                all_sheets_df['Issues of the Grange'] = new_sheet_df
        


        else:
            # If 'Issues of the Grange' is not present or 'Sentence' column is missing,
            # notify and skip processing related to it
            print("'Sentence' column not found in 'Issues of the Grange'. Skipping related processing.")
            
    
            ####################### FINAL SAVE #######################
            # Save all DataFrames to a new Excel file
            #output_file = "/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_sorted2.xlsx"
            #output_file = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script4/Ecchinswell4.xlsx'
                
                  
            with pd.ExcelWriter(output_file) as writer:
                for sheet_name, df in all_sheets_df.items():
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                            
            # Optionally add a print statement to confirm which file has been saved
            print(f"Saved processed data to {output_file}")

     

Check1: Columns in issues_df after initialization: Index(['Heading', 'Sentence', 'Quarters 1', 'Bushels 1', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servants & livestock Bushels',
       'Manorial servants & livestock Total Quarters', 'Sold Quarters',
       'Sold Bushels', 'Sold Total Quarters', 'Account Quarters',
       'Account Bushels', 'Account Total Quarters'],
      dtype='object')
Updated columns after processing:  Index(['Heading', 'Sentence', 'Quarters 1_x', 'Bushels 1_x', 'Sown Quarters',
       'Sown Bushels', 'Sown Total Quarters', 'Bought in Quarters',
       'Bought in Bushels', 'Bought in Total Quarters', 'Tithe Quarters',
       'Tithe Bushels', 'Tithe Total Quarters',
       'Manorial servants & livestock Quarters',
       'Manorial servan

In [25]:
# ======================= SCRIPT 4 — Working alternative =======================
import os
from pathlib import Path
import pandas as pd
import re

# ----------------------- CONFIG / PATHS -----------------------
document_names = [
    "Adderbury","Alresford","AlresfordBorough","Alverstoke","Ashmansworth",
    "Beauworth","Bentley","BishopsFonthill","BishopsSutton","BishopsWaltham",
    "Bitterne","Brightwell","Burghclere","Cams","Cheriton","Crawley","Culham",
    "Downton","DowntonBorough","Droxford","EastKnoyle","EastMeon","EastMeonChurch",
    "Ecchinswell","Esher","Farnham","Gosport","Hambledon","Harwell","Havant",
    "Highclere","HindonBorough","Holway","Ivinghoe","Kingston","Marshalsea",
    "Merdon","Morton","Newtown","NorthWaltham","Otterford","Overton",
    "OvertonBorough","Poundisford","Rimpton","SokeofWinchester","Southwark",
    "Staplegrove","StGilesFair","Taunton","TauntonBorough","TheChurch",
    "TrullBishopsHull","Twyford","Upton","WalthamStLawrence","Warfield","Wargrave",
    "Warren","WestWycombe","Wield","Witney","WitneyBorough","Wolvesey","Woodhay"
]

base_input_path  = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script3\\"
base_output_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script4\\"
Path(base_output_path).mkdir(parents=True, exist_ok=True)

# ----------------------- HELPERS -----------------------
def extract_specific_quarters_bushels(sentence: str, keywords):
    """Return (quarters, bushels) from a sentence iff any keyword is present."""
    if not isinstance(sentence, str) or not sentence:
        return 0.0, 0.0
    sent = sentence.lower()
    if not any(k in sent for k in keywords):
        return 0.0, 0.0
    matches = re.findall(r'(\d+(?:\.\d+)?)\s+quarters(?:\s+(\d+(?:\.\d+)?)\s+bushels)?', sentence, flags=re.IGNORECASE)
    if not matches:
        return 0.0, 0.0
    q = sum(float(qr) for qr, _ in matches if qr)
    b = sum(float(bs) for _, bs in matches if bs)
    return q, b

categories = {
    'Sown': ['sown'],
    'Bought in': ['bought'],
    'Tithe': ['tithe'],
    'Manorial servants & livestock': [
        'delivered to the door-keeper','provision of the steward',
        'provision of the constable','for the liveries of','fodder of'
    ],
    'Sold': ['sold'],
    'Account': ['same render account', 'from the whole issue', 'same renders account']
}

corn_types = [
    'wheat','mancorn','maslin','barley','first-grade malt','second-grade malt',
    'peas','oats','oatmeal','vetches','rye','dredge','malt','meal','curall'
]

# ----------------------- PROCESS ALL MANORS -----------------------
errors = []

for doc_name in document_names:
    input_file  = f"{base_input_path}{doc_name}3.xlsx"
    output_file = f"{base_output_path}{doc_name}4.xlsx"
    try:
        # ---- load ----
        all_sheets_df = pd.read_excel(input_file, sheet_name=None)
        if 'Issues of the Grange' not in all_sheets_df or 'raw data' not in all_sheets_df:
            raise KeyError("Expected sheets 'Issues of the Grange' and 'raw data' not found")

        issues_df   = all_sheets_df['Issues of the Grange'].copy()
        raw_data_df = all_sheets_df['raw data'].copy()

        if 'Sentence' not in issues_df.columns or 'Heading' not in issues_df.columns:
            print(f"Skipping {doc_name}: missing 'Heading' or 'Sentence' in Issues of the Grange")
            # still write out the unmodified workbook to keep parity
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                for s, df_out in all_sheets_df.items():
                    if isinstance(df_out, pd.DataFrame):
                        df_out.to_excel(writer, sheet_name=s[:31], index=False)
            continue

        # ---- drop money columns if present ----
        issues_df = issues_df.drop(columns=["Total in Pounds", "Pounds", "Shillings", "Pence"], errors='ignore')

        # ---- category extraction (one pass, robust) ----
        # ensure string columns to avoid weird types
        issues_df['Sentence'] = issues_df['Sentence'].astype(str)
        issues_df['Heading']  = issues_df['Heading'].astype(str)

        for cat, keys in categories.items():
            # series of (q, b) tuples
            q_b = issues_df['Sentence'].apply(lambda x: extract_specific_quarters_bushels(x, keys))

            # coerce every element to a 2-tuple; bad/None -> (0.0, 0.0)
            pairs = q_b.apply(
                lambda t: t if isinstance(t, (list, tuple)) and len(t) == 2
                else (0.0, 0.0)
            )

            # assign robustly without DataFrame constructor
            q_series = pairs.apply(
                lambda t: float(t[0]) if isinstance(t, (list, tuple)) and len(t) == 2 and pd.notna(t[0]) else 0.0
            )
            b_series = pairs.apply(
                lambda t: float(t[1]) if isinstance(t, (list, tuple)) and len(t) == 2 and pd.notna(t[1]) else 0.0
            )

            issues_df[f'{cat} Quarters'] = q_series.values
            issues_df[f'{cat} Bushels']  = b_series.values


            # totals and numeric cleanup
            issues_df[f'{cat} Total Quarters'] = (
                pd.to_numeric(issues_df[f'{cat} Quarters'], errors='coerce').fillna(0)
                + pd.to_numeric(issues_df[f'{cat} Bushels'], errors='coerce').fillna(0) / 8
            )
            for c in (f'{cat} Quarters', f'{cat} Bushels', f'{cat} Total Quarters'):
                issues_df[c] = pd.to_numeric(issues_df[c], errors='coerce').fillna(0)

        # ---- bought-only flag (vectorized) ----
        issues_df['bought only'] = (
            issues_df['Sentence'].str.contains('bought', case=False, na=False) &
            issues_df['Sentence'].str.contains('same render account|same renders account', case=False, na=False)
        ).astype(int)

        # ---- merge selective columns from raw_data_df to avoid _x/_y ----
        base_copy_cols = [
            "Total Quarters Except 1","Total Bushels Except 1",
            "Total Corn Cross-Check","Total Corn Manorial Account","Corn Quantity Error"
        ]
        # dynamically add Quarters i / Bushels i if they exist
        dyn_cols = []
        for i in range(1, 31):
            if f"Quarters {i}" in raw_data_df.columns: dyn_cols.append(f"Quarters {i}")
            if f"Bushels {i}" in raw_data_df.columns: dyn_cols.append(f"Bushels {i}")
        cols_to_merge = ["Heading","Sentence"] + [c for c in base_copy_cols + dyn_cols if c in raw_data_df.columns]
        issues_df = issues_df.merge(raw_data_df[cols_to_merge], on=["Heading","Sentence"], how="left")

        # ---- aggregate / build new Issues sheet ----
        bought_only_agg = (
            issues_df.groupby('Heading', as_index=False)['bought only'].max()
            .rename(columns={'bought only':'Bought Only Aggregated'})
        )
        bought_only_dict = dict(zip(bought_only_agg['Heading'], bought_only_agg['Bought Only Aggregated']))

        # start with one row per original heading
        new_rows = []
        for _, row in issues_df.iterrows():
            new_rows.append({
                'Gross Output': row['Heading'],
                'Bought Only Aggregated': bought_only_dict.get(row['Heading'], 0),
                'Account Total Quarters': row.get('Account Total Quarters', 0),
                'Bought in Total Quarters': row.get('Bought in Total Quarters', 0),
                'Sold Total Quarters': row.get('Sold Total Quarters', 0),
                'Account Quarters': row.get('Account Quarters', 0),
                'Account Bushels': row.get('Account Bushels', 0),
                'Bought in Quarters': row.get('Bought in Quarters', 0),
                'Bought in Bushels': row.get('Bought in Bushels', 0),
                'Sold Quarters': row.get('Sold Quarters', 0),
                'Sold Bushels': row.get('Sold Bushels', 0),
            })

        # ensure canonical corn rows exist even if absent in headings
        for ct in corn_types:
            if not any(str(r['Gross Output']).lower() == ct for r in new_rows):
                new_rows.append({
                    'Gross Output': ct.capitalize(),
                    'Bought Only Aggregated': int(bought_only_agg['Bought Only Aggregated']
                                                  [bought_only_agg['Heading'].str.lower()==ct].max()
                                                  if any(bought_only_agg['Heading'].str.lower()==ct) else 0),
                    'Account Total Quarters': 0,
                    'Bought in Total Quarters': 0,
                    'Sold Total Quarters': 0,
                    'Account Quarters': 0, 'Account Bushels':0,
                    'Bought in Quarters':0, 'Bought in Bushels':0,
                    'Sold Quarters':0, 'Sold Bushels':0
                })

        new_sheet_df = pd.DataFrame(new_rows)

        # normalize column names in case some came as "Total Sold/Bought Quarters"
        new_sheet_df = new_sheet_df.rename(columns={
            'Total Sold Quarters':'Sold Total Quarters',
            'Total Bought Quarters':'Bought in Total Quarters'
        })

        # coerce and fill numeric columns
        for col in [
            'Account Total Quarters','Bought in Total Quarters','Sold Total Quarters',
            'Account Quarters','Account Bushels',
            'Bought in Quarters','Bought in Bushels',
            'Sold Quarters','Sold Bushels'
        ]:
            if col not in new_sheet_df.columns:
                new_sheet_df[col] = 0
            new_sheet_df[col] = pd.to_numeric(new_sheet_df[col], errors='coerce').fillna(0)

        # collapse duplicates by corn label (keep max per metric)
        group_cols = ['Gross Output']
        agg_map = {c:'max' for c in new_sheet_df.columns if c not in group_cols}
        new_sheet_df = new_sheet_df.groupby(group_cols, as_index=False).agg(agg_map)

        # Total Corn Quantity (respect "bought only")
        new_sheet_df['Bought Only Aggregated'] = pd.to_numeric(
            new_sheet_df.get('Bought Only Aggregated', 0), errors='coerce'
        ).fillna(0).astype(int)

        new_sheet_df['Total Corn Quantity'] = new_sheet_df.apply(
            lambda r: r['Account Total Quarters'] if r['Bought Only Aggregated'] == 1
                      else r['Account Total Quarters'] + r['Bought in Total Quarters'],
            axis=1
        )
        new_sheet_df['Difference Produced+Bought-Sold'] = (
            new_sheet_df['Total Corn Quantity'] - new_sheet_df['Sold Total Quarters']
        )

        # consistent column order
        ordered = [
            'Gross Output','Bought Only Aggregated',
            'Account Quarters','Account Bushels','Account Total Quarters',
            'Bought in Quarters','Bought in Bushels','Bought in Total Quarters',
            'Sold Quarters','Sold Bushels','Sold Total Quarters',
            'Total Corn Quantity','Difference Produced+Bought-Sold'
        ]
        for c in ordered:
            if c not in new_sheet_df.columns: new_sheet_df[c] = 0
        new_sheet_df = new_sheet_df[ordered]

        # ---- PRICES: extract sale-of-corn rows & compute average prices (FIXED ALIGNMENT) ----
        sale_mask = raw_data_df['Heading'].astype(str).str.contains(
            r"Sale of Corn|wheat, rye, dredge, peas and oats sold", case=False, na=False, regex=True
        )
        sale_df = raw_data_df.loc[sale_mask].copy()

        if not sale_df.empty:
            base_cols = [
                'Heading','Sentence','Total in Pounds','Pounds','Shillings','Pence','Corn',
                'Total Quarters Except 1','Total Bushels Except 1',
                'Total Corn Cross-Check','Total Corn Manorial Account','Corn Quantity Error'
            ]

            # Collect any available banded columns (quantities + prices)
            dyn_cols = []
            for i in range(1, 31):
                for stem in (f'Quarters {i}', f'Bushels {i}', f'shilling a quarter {i}', f'pence a quarter {i}', f'£ a quarter {i}'):
                    if stem in sale_df.columns:
                        dyn_cols.append(stem)

            keep_cols = [c for c in base_cols + dyn_cols if c in sale_df.columns]
            sale_df = sale_df[keep_cols].copy()

            # 1) Make TOTALS explicit: if totals exist, standardize their names
            #    (Keep them at index 0, but we will NOT use index 0 in the weighted loop.)
            if 'Quarters 1' in sale_df.columns:
                sale_df = sale_df.rename(columns={'Quarters 1':'Quarters 0'})  # treat first occurrence as totals
            if 'Bushels 1' in sale_df.columns:
                sale_df = sale_df.rename(columns={'Bushels 1':'Bushels 0'})

            # 2) SHIFT *ALL* matched series (quantities AND prices) to preserve alignment
            def shift_series(prefix: str):
                # shift 2->1, 3->2, ... for any existing columns
                for i in range(30, 1, -1):  # reverse to avoid overwrite
                    old = f'{prefix} {i}'
                    new = f'{prefix} {i-1}'
                    if old in sale_df.columns:
                        sale_df.rename(columns={old: new}, inplace=True)

            shift_series('Quarters')
            shift_series('Bushels')
            shift_series('shilling a quarter')
            shift_series('pence a quarter')
            shift_series('£ a quarter')

            # 3) Build £/quarter where we have shilling/pence (aligned indices now)
            for i in range(0, 30):  # includes 0 (totals), but we'll only weight i>=1
                s_col, p_col, pound_col = f'shilling a quarter {i}', f'pence a quarter {i}', f'£ a quarter {i}'
                if s_col in sale_df.columns and p_col in sale_df.columns:
                    s = pd.to_numeric(sale_df[s_col], errors='coerce').fillna(0)
                    p = pd.to_numeric(sale_df[p_col], errors='coerce').fillna(0)
                    sale_df[pound_col] = s/20 + p/240

            # 4) Compute total quarters SOLD from totals row (0) if present
            if 'Quarters 0' in sale_df.columns and 'Bushels 0' in sale_df.columns:
                sale_df['Quarters Sold'] = pd.to_numeric(sale_df['Quarters 0'], errors='coerce').fillna(0) + \
                                        pd.to_numeric(sale_df['Bushels 0'],  errors='coerce').fillna(0)/8
            else:
                sale_df['Quarters Sold'] = 0.0

            # 5) If there are no band rows (i>=1) but we have totals + total value, create a synthetic band 1
            def has_any_band(row):
                return any((f'Quarters {i}' in row.index and pd.notna(row[f'Quarters {i}'])) or
                        (f'Bushels {i}'  in row.index and pd.notna(row[f'Bushels {i}'])) or
                        (f'£ a quarter {i}' in row.index and pd.notna(row[f'£ a quarter {i}']))
                        for i in range(1, 30))

            no_band_mask = ~sale_df.apply(has_any_band, axis=1)
            # Create band 1 by copying totals into index 1 for those rows
            for col0, col1 in (('Quarters 0','Quarters 1'), ('Bushels 0','Bushels 1')):
                if col0 in sale_df.columns:
                    sale_df.loc[no_band_mask, col1] = sale_df.loc[no_band_mask, col0]
            # If we still don't have a price at 1, backfill from total value / quarters sold
            # (behaves like a single-band price)
            if '£ a quarter 1' not in sale_df.columns:
                sale_df['£ a quarter 1'] = pd.NA
            with pd.option_context('mode.use_inf_as_na', True):
                sale_df.loc[no_band_mask & (sale_df['Quarters Sold'] > 0), '£ a quarter 1'] = \
                    pd.to_numeric(sale_df['Total in Pounds'], errors='coerce') / sale_df['Quarters Sold']

            # 6) Weighted & unweighted averages (use ONLY bands i>=1 so we don't double-count totals)
            def weighted_avg(row):
                wsum = 0.0; qtot = 0.0; nprices = 0
                for i in range(1, 30):
                    pc, qc, bc = f'£ a quarter {i}', f'Quarters {i}', f'Bushels {i}'
                    if pc in row.index and pd.notna(row[pc]):
                        q = (pd.to_numeric(row.get(qc), errors='coerce') if qc in row.index else 0)
                        b = (pd.to_numeric(row.get(bc), errors='coerce') if bc in row.index else 0)
                        q = (0 if pd.isna(q) else q) + (0 if pd.isna(b) else b)/8
                        if q > 0:
                            wsum += float(row[pc]) * float(q)
                            qtot += float(q)
                            nprices += 1
                if nprices == 1 and '£ a quarter 1' in row.index and pd.notna(row['£ a quarter 1']):
                    return float(row['£ a quarter 1'])
                return (wsum / qtot) if qtot else pd.NA

            def unweighted_avg(row):
                vals = []
                for i in range(1, 30):
                    pc = f'£ a quarter {i}'
                    if pc in row.index and pd.notna(row[pc]):
                        vals.append(float(row[pc]))
                return (sum(vals)/len(vals)) if vals else pd.NA

            sale_df['Weighted Average Price per Quarter']     = sale_df.apply(weighted_avg, axis=1)
            sale_df['Non-Weighted Average Price per Quarter'] = sale_df.apply(unweighted_avg, axis=1)

            # 7) Merge onto Issues-of-the-Grange by corn label
            new_sheet_df['__lo'] = new_sheet_df['Gross Output'].astype(str).str.lower()
            if 'Corn' in sale_df.columns:
                sale_df['__lo_corn'] = sale_df['Corn'].astype(str).str.lower()
            else:
                sale_df['__lo_corn'] = ''
            sale_df = sale_df.drop_duplicates(subset=['__lo_corn'], keep='first')

            merged = pd.merge(
                new_sheet_df,
                sale_df[['__lo_corn','Weighted Average Price per Quarter','Non-Weighted Average Price per Quarter']],
                left_on='__lo', right_on='__lo_corn', how='left'
            ).drop(columns=['__lo','__lo_corn'], errors='ignore')

            # 8) Production values from the weighted/unweighted prices
            merged['Value of Total Production weighted']   = merged['Account Total Quarters'] * merged['Weighted Average Price per Quarter']
            merged['Value of Total Production unweighted'] = merged['Account Total Quarters'] * merged['Non-Weighted Average Price per Quarter']

            new_sheet_df = merged


        # ---- put sheets back and stamp version ----
        all_sheets_df['Issues of the Grange'] = new_sheet_df
        # keep a raw snapshot too, if you want it:
        all_sheets_df['Issues of the Grange raw'] = issues_df

        # provenance stamp (helps verify file came from this script run)
        all_sheets_df['Issues of the Grange']['_script4_stamp'] = pd.Timestamp.now().isoformat()

        # ---- write workbook ----
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            for sheet_name, df_out in all_sheets_df.items():
                if isinstance(df_out, pd.DataFrame):
                    df_out.to_excel(writer, sheet_name=sheet_name[:31], index=False)

        print(f"✅ {doc_name}: saved -> {output_file}")

    except Exception as e:
        errors.append((doc_name, str(e)))
        print(f"❌ {doc_name}: {e}")

print("\nDone.")
if errors:
    print("Some documents failed:")
    for n, msg in errors:
        print(f" - {n}: {msg}")
# ======================= end cell =======================


❌ Adderbury: arg must be a list, tuple, 1-d array, or Series
❌ Alresford: arg must be a list, tuple, 1-d array, or Series
Skipping AlresfordBorough: missing 'Heading' or 'Sentence' in Issues of the Grange
Skipping Alverstoke: missing 'Heading' or 'Sentence' in Issues of the Grange
❌ Ashmansworth: arg must be a list, tuple, 1-d array, or Series
✅ Beauworth: saved -> C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script4\\Beauworth4.xlsx
❌ Bentley: arg must be a list, tuple, 1-d array, or Series
❌ BishopsFonthill: arg must be a list, tuple, 1-d array, or Series
✅ BishopsSutton: saved -> C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script4\\BishopsSutton4.xlsx
❌ BishopsWaltham: arg must be a list, tuple, 1-d array, or Series
❌ Bitterne: arg must be a list, tuple, 1-d array, or Series
❌ Br

In [32]:
# ======================= SCRIPT 4 — Robust end-to-end (fixed) =======================
# Processes all manor workbooks:
# 1) Parses Issues of the Grange to compute per-category quantities
# 2) Builds a clean "Issues of the Grange" summary sheet
# 3) Extracts sale-of-corn lines to compute weighted/unweighted prices
# 4) Merges prices into the Issues summary and computes production values
# Always writes an output file (even if some inputs are incomplete).
# ===========================================================================
# --- add this near the top with your other imports ---
import traceback

import os
from pathlib import Path
import pandas as pd
import re

# ----------------------- CONFIG / PATHS -----------------------
document_names = [
    "Adderbury","Alresford","AlresfordBorough","Alverstoke","Ashmansworth",
    "Beauworth","Bentley","BishopsFonthill","BishopsSutton","BishopsWaltham",
    "Bitterne","Brightwell","Burghclere","Cams","Cheriton","Crawley","Culham",
    "Downton","DowntonBorough","Droxford","EastKnoyle","EastMeon","EastMeonChurch",
    "Ecchinswell","Esher","Farnham","Gosport","Hambledon","Harwell","Havant",
    "Highclere","HindonBorough","Holway","Ivinghoe","Kingston","Marshalsea",
    "Merdon","Morton","Newtown","NorthWaltham","Otterford","Overton",
    "OvertonBorough","Poundisford","Rimpton","SokeofWinchester","Southwark",
    "Staplegrove","StGilesFair","Taunton","TauntonBorough","TheChurch",
    "TrullBishopsHull","Twyford","Upton","WalthamStLawrence","Warfield","Wargrave",
    "Warren","WestWycombe","Wield","Witney","WitneyBorough","Wolvesey","Woodhay"
]

base_input_path  = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script3\\"
base_output_path = r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script4\\"
Path(base_output_path).mkdir(parents=True, exist_ok=True)

# ----------------------- CONSTANTS -----------------------
categories = {
    'Sown': ['sown'],
    'Bought in': ['bought'],
    'Tithe': ['tithe'],
    'Manorial servants & livestock': [
        'delivered to the door-keeper','provision of the steward',
        'provision of the constable','for the liveries of','fodder of'
    ],
    'Sold': ['sold'],
    'Account': ['same render account', 'from the whole issue', 'same renders account']
}

corn_types = [
    'wheat','mancorn','maslin','barley','first-grade malt','second-grade malt',
    'peas','oats','oatmeal','vetches','rye','dredge','malt','meal','curall'
]

# ----------------------- HELPERS -----------------------
def extract_specific_quarters_bushels(sentence: str, keywords):
    """Return (quarters, bushels) from a sentence iff any keyword is present; else (0,0)."""
    if not isinstance(sentence, str) or not sentence:
        return (0.0, 0.0)
    sent = sentence.lower()
    if not any(k in sent for k in keywords):
        return (0.0, 0.0)
    matches = re.findall(
        r'(\d+(?:\.\d+)?)\s+quarters(?:\s+(\d+(?:\.\d+)?)\s+bushels)?',
        sentence,
        flags=re.IGNORECASE
    )
    if not matches:
        return (0.0, 0.0)
    q = sum(float(qr) for qr, _ in matches if qr)
    b = sum(float(bs) for _, bs in matches if bs)
    return (q, b)

def safe_num(x, default=0.0):
    """
    Coerces Series OR scalar to numeric.
    - Series/array-like: returns Series with NaNs filled.
    - Scalar: returns float with NaNs -> default.
    """
    try:
        out = pd.to_numeric(x, errors='coerce')
        if hasattr(out, "fillna"):
            return out.fillna(default)
        return float(out) if pd.notna(out) else float(default)
    except Exception:
        try:
            return float(x)
        except Exception:
            return float(default)

# ----------------------- MAIN -----------------------
errors = []

for doc_name in document_names:
    input_file  = f"{base_input_path}{doc_name}3.xlsx"
    output_file = f"{base_output_path}{doc_name}4.xlsx"

    try:
        # ---- load whole workbook ----
        all_sheets_df = pd.read_excel(input_file, sheet_name=None)
        if not isinstance(all_sheets_df, dict):
            raise ValueError("Workbook read did not return a dict of DataFrames.")

        # if required sheets missing, still write the workbook out unchanged (parity)
        if 'Issues of the Grange' not in all_sheets_df or 'raw data' not in all_sheets_df:
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                for s, df_out in all_sheets_df.items():
                    if isinstance(df_out, pd.DataFrame):
                        df_out.to_excel(writer, sheet_name=s[:31], index=False)
            print(f"⚠️ {doc_name}: missing sheet(s) — wrote unchanged workbook.")
            continue

        issues_df   = all_sheets_df['Issues of the Grange'].copy()
        raw_data_df = all_sheets_df['raw data'].copy()

        # If core columns missing, write unchanged workbook
        if 'Sentence' not in issues_df.columns or 'Heading' not in issues_df.columns:
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                for s, df_out in all_sheets_df.items():
                    if isinstance(df_out, pd.DataFrame):
                        df_out.to_excel(writer, sheet_name=s[:31], index=False)
            print(f"⚠️ {doc_name}: 'Heading'/'Sentence' missing — wrote unchanged workbook.")
            continue

        # ---- basic cleanup ----
        issues_df = issues_df.drop(columns=["Total in Pounds", "Pounds", "Shillings", "Pence"], errors='ignore')
        issues_df['Sentence'] = issues_df['Sentence'].astype(str)
        issues_df['Heading']  = issues_df['Heading'].astype(str)

        # ---- category extraction (robust) ----
        for cat, keys in categories.items():
            res = issues_df['Sentence'].apply(lambda x: extract_specific_quarters_bushels(x, keys))
            q_series = res.apply(lambda t: float(t[0]) if isinstance(t, (list, tuple)) and len(t) == 2 and pd.notna(t[0]) else 0.0)
            b_series = res.apply(lambda t: float(t[1]) if isinstance(t, (list, tuple)) and len(t) == 2 and pd.notna(t[1]) else 0.0)

            issues_df[f'{cat} Quarters'] = q_series.to_numpy()
            issues_df[f'{cat} Bushels']  = b_series.to_numpy()
            issues_df[f'{cat} Total Quarters'] = safe_num(q_series) + safe_num(b_series)/8

        # ---- bought-only flag ----
        issues_df['bought only'] = (
            issues_df['Sentence'].str.contains('bought', case=False, na=False) &
            issues_df['Sentence'].str.contains('same render account|same renders account', case=False, na=False)
        ).astype(int)

        # ---- merge selective columns (avoid _x/_y) ----
        base_copy_cols = [
            "Total Quarters Except 1","Total Bushels Except 1",
            "Total Corn Cross-Check","Total Corn Manorial Account","Corn Quantity Error"
        ]
        dyn_cols = []
        for i in range(1, 31):
            if f"Quarters {i}" in raw_data_df.columns: dyn_cols.append(f"Quarters {i}")
            if f"Bushels {i}" in raw_data_df.columns: dyn_cols.append(f"Bushels {i}")
        cols_to_merge = ["Heading","Sentence"] + [c for c in base_copy_cols + dyn_cols if c in raw_data_df.columns]
        issues_df = issues_df.merge(raw_data_df[cols_to_merge], on=["Heading","Sentence"], how="left")

        # ===================== FIXED BLOCK (no Series truth checks) =====================
        # Build new Issues-of-the-Grange (summary)
        bought_only_agg = (
            issues_df.groupby('Heading', as_index=False)['bought only'].max()
            .rename(columns={'bought only':'Bought Only Aggregated'})
        )
        bought_only_dict = dict(zip(bought_only_agg['Heading'], bought_only_agg['Bought Only Aggregated']))

        # initial rows (one per original heading)
        rows = []
        for _, r in issues_df.iterrows():
            rows.append({
                'Gross Output': r['Heading'],
                'Bought Only Aggregated': bought_only_dict.get(r['Heading'], 0),
                'Account Total Quarters': r.get('Account Total Quarters', 0),
                'Bought in Total Quarters': r.get('Bought in Total Quarters', 0),
                'Sold Total Quarters': r.get('Sold Total Quarters', 0),
                'Account Quarters': r.get('Account Quarters', 0),
                'Account Bushels': r.get('Account Bushels', 0),
                'Bought in Quarters': r.get('Bought in Quarters', 0),
                'Bought in Bushels': r.get('Bought in Bushels', 0),
                'Sold Quarters': r.get('Sold Quarters', 0),
                'Sold Bushels': r.get('Sold Bushels', 0),
            })

        # ===================== REPLACE this whole block =====================
        # ensure canonical corn rows exist even if absent in headings
        # (robust: never branch directly on a Series)
        rows_lower = set()
        for r in rows:
            try:
                rows_lower.add(str(r.get('Gross Output', '')).lower())
            except Exception:
                # if anything weird, just skip that entry
                continue

        for ct in corn_types:
            if ct not in rows_lower:
                # Build the mask as a Series, then only branch on the scalar .empty
                mask = bought_only_agg['Heading'].astype(str).str.lower().eq(ct)
                subset = bought_only_agg.loc[mask, 'Bought Only Aggregated']
                # derive a scalar safely
                if subset.empty:
                    bo_val = 0
                else:
                    bo_val = pd.to_numeric(subset, errors='coerce').fillna(0).max()
                    bo_val = 0 if pd.isna(bo_val) else int(bo_val)

                rows.append({
                    'Gross Output': ct.capitalize(),
                    'Bought Only Aggregated': bo_val,
                    'Account Total Quarters': 0,
                    'Bought in Total Quarters': 0,
                    'Sold Total Quarters': 0,
                    'Account Quarters': 0, 'Account Bushels': 0,
                    'Bought in Quarters': 0, 'Bought in Bushels': 0,
                    'Sold Quarters': 0, 'Sold Bushels': 0
                })
        # ===================== END replacement =====================

        # create DataFrame from the SAME list we appended to
        new_sheet_df = pd.DataFrame(rows).rename(columns={
            'Total Sold Quarters':'Sold Total Quarters',
            'Total Bought Quarters':'Bought in Total Quarters'
        })

        # numeric coercions
        numeric_cols = [
            'Account Total Quarters','Bought in Total Quarters','Sold Total Quarters',
            'Account Quarters','Account Bushels',
            'Bought in Quarters','Bought in Bushels',
            'Sold Quarters','Sold Bushels','Bought Only Aggregated'
        ]
        for c in numeric_cols:
            if c not in new_sheet_df.columns:
                new_sheet_df[c] = 0
            new_sheet_df[c] = safe_num(new_sheet_df[c], default=0)

        # collapse duplicates by Gross Output (max aggregator)
        agg_map = {c: 'max' for c in new_sheet_df.columns if c != 'Gross Output'}
        new_sheet_df = new_sheet_df.groupby('Gross Output', as_index=False).agg(agg_map)

        # totals/differences
        new_sheet_df['Bought Only Aggregated'] = new_sheet_df['Bought Only Aggregated'].astype(int)
        new_sheet_df['Total Corn Quantity'] = new_sheet_df.apply(
            lambda r: r['Account Total Quarters'] if r['Bought Only Aggregated'] == 1
                      else r['Account Total Quarters'] + r['Bought in Total Quarters'],
            axis=1
        )
        new_sheet_df['Difference Produced+Bought-Sold'] = new_sheet_df['Total Corn Quantity'] - new_sheet_df['Sold Total Quarters']

        # final order
        ordered = [
            'Gross Output','Bought Only Aggregated',
            'Account Quarters','Account Bushels','Account Total Quarters',
            'Bought in Quarters','Bought in Bushels','Bought in Total Quarters',
            'Sold Quarters','Sold Bushels','Sold Total Quarters',
            'Total Corn Quantity','Difference Produced+Bought-Sold'
        ]
        for c in ordered:
            if c not in new_sheet_df.columns:
                new_sheet_df[c] = 0
        new_sheet_df = new_sheet_df[ordered]
        # ===================== /FIXED BLOCK =====================

        # ---- PRICES (sale of corn) ----
        sale_mask = raw_data_df['Heading'].astype(str).str.contains(
            r"Sale of Corn|wheat, rye, dredge, peas and oats sold", case=False, na=False, regex=True
        )
        sale_df = raw_data_df.loc[sale_mask].copy()

        if not sale_df.empty:
            base_cols = [
                'Heading','Sentence','Total in Pounds','Pounds','Shillings','Pence','Corn',
                'Total Quarters Except 1','Total Bushels Except 1',
                'Total Corn Cross-Check','Total Corn Manorial Account','Corn Quantity Error'
            ]
            dyn_cols = []
            for i in range(1, 31):
                for stem in (f'Quarters {i}', f'Bushels {i}', f'shilling a quarter {i}', f'pence a quarter {i}', f'£ a quarter {i}'):
                    if stem in sale_df.columns:
                        dyn_cols.append(stem)
            keep_cols = [c for c in base_cols + dyn_cols if c in sale_df.columns]
            sale_df = sale_df[keep_cols].copy()

            # Standardize totals as index 0; then shift all series to preserve price/qty alignment
            if 'Quarters 1' in sale_df.columns: sale_df.rename(columns={'Quarters 1':'Quarters 0'}, inplace=True)
            if 'Bushels 1' in sale_df.columns:  sale_df.rename(columns={'Bushels 1':'Bushels 0'}, inplace=True)

            def shift_series(prefix: str):
                for i in range(30, 1, -1):
                    old, new = f'{prefix} {i}', f'{prefix} {i-1}'
                    if old in sale_df.columns:
                        sale_df.rename(columns={old: new}, inplace=True)

            shift_series('Quarters')
            shift_series('Bushels')
            shift_series('shilling a quarter')
            shift_series('pence a quarter')
            shift_series('£ a quarter')

            # Build £/quarter where shilling/pence available
            for i in range(0, 30):
                s_col, p_col, pound_col = f'shilling a quarter {i}', f'pence a quarter {i}', f'£ a quarter {i}'
                if s_col in sale_df.columns and p_col in sale_df.columns:
                    s = safe_num(sale_df[s_col], 0)
                    p = safe_num(sale_df[p_col], 0)
                    sale_df[pound_col] = s/20 + p/240

            # Quarters Sold from totals
            if 'Quarters 0' in sale_df.columns and 'Bushels 0' in sale_df.columns:
                sale_df['Quarters Sold'] = safe_num(sale_df['Quarters 0'], 0) + safe_num(sale_df['Bushels 0'], 0)/8
            else:
                sale_df['Quarters Sold'] = 0.0

            # If no band rows but totals+value exist, synthesize band 1
            def has_any_band(row):
                found = False
                for i in range(1, 30):
                    qc, bc, pc = f'Quarters {i}', f'Bushels {i}', f'£ a quarter {i}'
                    if (qc in row.index and pd.notna(row.get(qc))) or \
                       (bc in row.index and pd.notna(row.get(bc))) or \
                       (pc in row.index and pd.notna(row.get(pc))):
                        found = True
                        break
                return found
            no_band_mask = ~sale_df.apply(has_any_band, axis=1)
            for col0, col1 in (('Quarters 0','Quarters 1'), ('Bushels 0','Bushels 1')):
                if col0 in sale_df.columns:
                    sale_df.loc[no_band_mask, col1] = sale_df.loc[no_band_mask, col0]
            if '£ a quarter 1' not in sale_df.columns:
                sale_df['£ a quarter 1'] = pd.NA
            # compute price from total value / total qty when needed
            sale_df.loc[no_band_mask & (sale_df['Quarters Sold'] > 0), '£ a quarter 1'] = \
                safe_num(sale_df['Total in Pounds'], 0) / sale_df['Quarters Sold']

            # Weighted / unweighted averages (use ONLY bands >=1)
            def weighted_avg(row):
                wsum = 0.0
                qtot = 0.0
                nprices = 0
                for i in range(1, 30):
                    pc, qc, bc = f'£ a quarter {i}', f'Quarters {i}', f'Bushels {i}'
                    price = row.get(pc, pd.NA)
                    if pd.notna(price):
                        q = safe_num(row.get(qc, 0), 0.0)
                        b = safe_num(row.get(bc, 0), 0.0)
                        qty = (q if pd.notna(q) else 0.0) + (b if pd.notna(b) else 0.0) / 8.0
                        if qty > 0:
                            wsum += float(price) * float(qty)
                            qtot += float(qty)
                            nprices += 1
                if nprices == 1 and pd.notna(row.get('£ a quarter 1', pd.NA)):
                    return float(row['£ a quarter 1'])
                return (wsum / qtot) if qtot else pd.NA

            def unweighted_avg(row):
                vals = []
                for i in range(1, 30):
                    pc = f'£ a quarter {i}'
                    price = row.get(pc, pd.NA)
                    if pd.notna(price):
                        vals.append(float(price))
                return (sum(vals) / len(vals)) if vals else pd.NA

            sale_df['Weighted Average Price per Quarter']     = sale_df.apply(weighted_avg, axis=1)
            sale_df['Non-Weighted Average Price per Quarter'] = sale_df.apply(unweighted_avg, axis=1)

            # Merge onto Issues by corn label (case-insensitive)
            new_sheet_df['__lo'] = new_sheet_df['Gross Output'].astype(str).str.lower()
            if 'Corn' in sale_df.columns:
                sale_df['__lo_corn'] = sale_df['Corn'].astype(str).str.lower()
            else:
                sale_df['__lo_corn'] = ''
            sale_df = sale_df.drop_duplicates(subset=['__lo_corn'], keep='first')

            merged = pd.merge(
                new_sheet_df,
                sale_df[['__lo_corn','Weighted Average Price per Quarter','Non-Weighted Average Price per Quarter']],
                left_on='__lo', right_on='__lo_corn', how='left'
            ).drop(columns=['__lo','__lo_corn'], errors='ignore')

            # Production values from the weighted/unweighted prices
            merged['Value of Total Production weighted']   = merged['Account Total Quarters'] * merged['Weighted Average Price per Quarter']
            merged['Value of Total Production unweighted'] = merged['Account Total Quarters'] * merged['Non-Weighted Average Price per Quarter']

            new_sheet_df = merged

        # ---- put sheets back; keep also a raw snapshot ----
        all_sheets_df['Issues of the Grange'] = new_sheet_df
        all_sheets_df['Issues of the Grange raw'] = issues_df
        all_sheets_df['Issues of the Grange']['_script4_stamp'] = pd.Timestamp.now().isoformat()

        # ---- write workbook (always) ----
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            for sheet_name, df_out in all_sheets_df.items():
                if isinstance(df_out, pd.DataFrame):
                    df_out.to_excel(writer, sheet_name=sheet_name[:31], index=False)

        print(f"✅ {doc_name}: saved -> {output_file}")

    except Exception as e:
        # If anything goes wrong, try to still write out the original workbook for parity
        try:
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                for s, df_out in all_sheets_df.items():
                    if isinstance(df_out, pd.DataFrame):
                        df_out.to_excel(writer, sheet_name=s[:31], index=False)
            print(f"❌ {doc_name}: {e} — wrote unchanged workbook.")
        except Exception as e2:
            print(f"❌ {doc_name}: {e} — also failed to write unchanged: {e2}")
        errors.append((doc_name, str(e)))

print("\nDone.")
if errors:
    print("Some documents had issues:")
    for n, msg in errors:
        print(f" - {n}: {msg}")
# ======================= end script =======================



❌ Adderbury: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). — wrote unchanged workbook.
❌ Alresford: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). — wrote unchanged workbook.
⚠️ AlresfordBorough: 'Heading'/'Sentence' missing — wrote unchanged workbook.
⚠️ Alverstoke: 'Heading'/'Sentence' missing — wrote unchanged workbook.
❌ Ashmansworth: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). — wrote unchanged workbook.
✅ Beauworth: saved -> C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\OCR 1301-1302\python transcription\script4\\Beauworth4.xlsx
❌ Bentley: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). — wrote unchanged workbook.
❌ BishopsFonthill: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all

In [205]:
####################### SCRIPT 5  ####################### 
import pandas as pd
import re
import numpy as np 


# This script populates the Stock sheet correctly


# Define document names and base paths
document_names = [
    "Adderbury", "Alresford", "AlresfordBorough", "Alverstoke", "Ashmansworth",
    "Beauworth", "Bentley", "BishopsFonthill", "BishopsSutton", "BishopsWaltham",
    "Bitterne", "Brightwell", "Burghclere", "Cams", "Cheriton", "Crawley", "Culham",
    "Downton", "DowntonBorough", "Droxford", "EastKnoyle", "EastMeon", "EastMeonChurch",
    "Ecchinswell", "Esher", "Farnham", "Gosport", "Hambledon", "Harwell", "Havant",
    "Highclere", "HindonBorough", "Holway", "Ivinghoe", "Kingston", "Marshalsea",
    "Merdon", "Morton", "Newtown", "NorthWaltham", "Otterford", "Overton",
    "OvertonBorough", "Poundisford", "Rimpton", "SokeofWinchester", "Southwark",
    "Staplegrove", "StGilesFair", "Taunton", "TauntonBorough", "TheChurch",
    "TrullBishopsHull", "Twyford", "Upton", "WalthamStLawrence", "Warfield", "Wargrave",
    "Warren", "WestWycombe", "Wield", "Witney", "WitneyBorough", "Wolvesey", "Woodhay"
]


# Paths setup
stock_file_path = "/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/stock.xlsx"
input_base_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script4/'
output_base_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script5/'

# Load the stock Excel sheet
#stock_file_path = "/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/stock.xlsx"
#stock_file_path = "/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/stock.xlsx"

stock_df = pd.read_excel(stock_file_path, sheet_name='stock')

# Path to the original Excel file
#input_file_path = "/Users/victoriagierok/Dropbox/My Mac (MacBook-Pro.fritz.box)/Desktop/Manorial1301_sorted2.xlsx"
#input_file_path = '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script4/Alverstoke4.xlsx'




##### Mapping all the types of Stock using Stock Excel
# Define the function to find and map "Stock" based on "Item" matches before a colon in "Sentence"
def find_stock(sentence):
    for _, row in stock_df.iterrows():
        item, stock_value = row['Item'], row['Stock']
        if re.search(rf'\b{re.escape(item)}\b(?=[:])', sentence, re.IGNORECASE):
            return stock_value
    return None


# Process each document
for doc_name in document_names:
    input_file_path = f"{input_base_path}{doc_name}4.xlsx"
    output_file_path = f"{output_base_path}{doc_name}5.xlsx"

    # Load all sheets into a dictionary of DataFrames
    all_sheets_df = pd.read_excel(input_file_path, sheet_name=None)


    if 'Stock' in all_sheets_df:
        stock_sheet_df = all_sheets_df['Stock']

    # Check if 'Stock' sheet exists and is not empty
    if 'Stock' in all_sheets_df and not all_sheets_df['Stock'].empty:
        stock_sheet_df = all_sheets_df['Stock']
        stock_sheet_df['stock'] = stock_sheet_df['Sentence'].apply(find_stock)
        stock_sheet_df['stock'] = stock_sheet_df['stock'].ffill()  # Forward fill 'stock' column
    
         # Drop unnecessary columns
        columns_to_delete = ["Total in Pounds", "Pounds", "Shillings", "Pence"]
        stock_sheet_df.drop(columns=columns_to_delete, inplace=True, errors='ignore')
    
        #### Extracting the numbers of various animals from the Sentence
        # Function to directly extract numbers for "added" and "murrain"
        def extract_number_before_keyword(sentence, keywords):
            for keyword in keywords:
            # Adjust the regex pattern to account for possible words between the number and the keyword
            # and to make sure the keyword is not followed by a full-stop or colon
                match = re.search(r'(\d+)\s+[^\.,;]*?\b' + re.escape(keyword) + r'\b', sentence)
                if match:
                    return match.group(1)
            return np.nan
    
        # Adjusted function for "inherited stock"
        def adjust_inherited_stock(row, found_matches):
            stock_type = row['stock']
            sentence = row['Sentence']
            # Only process if this stock type hasn't been matched yet
            if not found_matches.get(stock_type, False):
                match = re.search(r'(\d+)\s+[^\.,;]*?\bremain\b', sentence)
                if match:
                    found_matches[stock_type] = True  # Mark this stock type as matched
                    return match.group(1)
            return np.nan
    
        # Process other columns as necessary
        found_matches = {}  # Track matches for unique processing
        stock_sheet_df['inherited stock'] = stock_sheet_df.apply(lambda row: adjust_inherited_stock(row, found_matches), axis=1)
        stock_sheet_df['added'] = stock_sheet_df['Sentence'].apply(lambda x: extract_number_before_keyword(x, ["bought", "offspring"]))
        stock_sheet_df['murrain'] = stock_sheet_df['Sentence'].apply(lambda x: extract_number_before_keyword(x, ["in murrain"]))
        stock_sheet_df['sold'] = stock_sheet_df['Sentence'].apply(lambda x: extract_number_before_keyword(x, ["sold"]))
    
        # Update the dictionary with modified DataFrame
        all_sheets_df['Stock'] = stock_sheet_df
    else:
        print("The 'Stock' sheet is empty or does not exist. No processing will be applied to it.")
    
    

    # Save all DataFrames (sheets) to the new Excel file
    with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
        for sheet_name, df in all_sheets_df.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print("All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.")


All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.
All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.
The 'Stock' sheet is empty or does not exist. No processing will be applied to it.
All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.
The 'Stock' sheet is empty or does not exist. No processing will be applied to it.
All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.
All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.
All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.
All sheets, including the modified 'Stock' sheet, have been saved to the new file successfully.
The 'Stock' sheet is empty or does not exist. No processing will be applied to it.
All sheets, including the modified 'Stock' sheet, have been saved to the new fi

FileNotFoundError: [Errno 2] No such file or directory: '/Users/victoriagierok/Dropbox/Physical Capital England/Decennial 2018/Detailed Estimates/Winchester Pipe Rolls/1301-1302/python transcription/script4/TheChurch4.xlsx'

In [ ]:
####################### SCRIPT 6  ####################### 

# This script create the Overview sheet - populating it with the information from the other sheets